# Creating Coastal Watersheds for Lake Huron Coastal Wetlands (CW)

This notebook delineates **coastal watersheds** for **Lake Huron–connected coastal wetlands** under four inundation scenarios (**avg, low, high, surge**) while ensuring a **stable wetland identifier (`CW_Id`) is preserved throughout the full workflow**. The goal is to produce **one coastal watershed polygon per wetland (`CW_Id`)**, along with consistent wetland and watershed attributes needed for later merging and plotting (area + centroid coordinates).

---

## Key idea: keep `CW_Id` stable from start to finish
Raster-based watershed tools can replace feature IDs with raster values (e.g., `gridcode`) and can also split features during polygon/raster conversions. To avoid ID mismatches, this workflow:

- assigns and carries a stable **`CW_Id`** in all wetland layers,
- creates **one pour point per wetland** (inside the polygon),
- snaps pour points to the drainage network,
- delineates watersheds using the snapped points,
- **converts watershed outputs back to polygons** and **dissolves by `CW_Id`** so each wetland ends with **exactly one** watershed polygon.

---

## Inputs
Main inputs used in this notebook:

- **Coastal wetland polygons** (avg/low/high/surge inundation layers)
- **Shoreline polyline** (Lake Huron US-side shoreline)
- **Great Lakes Basin streams** (used to remove riparian/stream-connected wetland overlap)
- **D8 flow direction raster** (hydrologic routing grid)
- **Stream-watershed polygons** (areas draining to streams; removed from coastal watersheds)
- **Lake Huron polygon** (removed from final watershed polygons)

All distance-based operations are performed in **Great Lakes Albers (EPSG:3174)** (meters).

---

## Outputs (per inundation scenario)
For each scenario (**avg, low, high, surge**), the notebook produces:

### Wetland-side products
- **shoreline-interacting wetlands** (wetlands intersecting a 2000 m shoreline buffer)
- **riparian-erased wetlands** (wetlands with 50 m stream-buffer overlap removed)
- wetland attributes:
  - `CW_Id` (stable wetland identifier)
  - `CW_Area_m2`
  - wetland centroid coordinates in EPSG:3174 (`CW_cx`, `CW_cy`)
  - wetland centroid coordinates in WGS84 (`CW_lon`, `CW_lat`)

### Watershed-side products
- **pour points** (`*_pourpoints.shp`) — one point inside each wetland polygon
- **snapped pour points** — pour points snapped to a drainage cell using flow accumulation
- **watershed raster** (cell values correspond to `CW_Id`)
- **watershed polygons**, dissolved by `CW_Id` (one watershed per wetland)
- final coastal watershed polygons with attributes:
  - `CW_Id` (matching wetland `CW_Id`)
  - `WatershedArea_m2`
  - watershed centroid coordinates in EPSG:3174 (`WS_cx`, `WS_cy`)
  - watershed centroid coordinates in WGS84 (`WS_lon`, `WS_lat`)

---

## Workflow summary
For each inundation scenario (**avg/low/high/surge**):

1. **Assign stable IDs**
   - Ensure wetland polygons contain `CW_Id` and (optionally) `Coastal_Id` derived from `CW_Id`.

2. **Select shoreline-interacting wetlands**
   - Project shoreline to EPSG:3174, buffer by **2000 m**, and intersect with wetlands.

3. **Remove riparian/stream overlap**
   - Buffer streams by **50 m** and erase from the shoreline-interacting wetlands.

4. **Create pour points**
   - Dissolve wetlands by `CW_Id` and create one **inside point** per wetland (`*_pourpoints.shp`).

5. **Snap pour points**
   - Snap pour points to the drainage network using **SnapPourPoint** with flow accumulation.

6. **Delineate watersheds**
   - Use **Watershed** with the D8 flow direction raster and snapped pour points.

7. **Convert to polygons + enforce 1 watershed per wetland**
   - Convert watershed raster to polygons, set `CW_Id = gridcode`, and **dissolve by `CW_Id`**.

8. **Remove non-coastal drainage + lake area**
   - Erase stream-watershed polygons, then erase Lake Huron polygon from the watershed polygons.

9. **Compute areas + centroids**
   - Add watershed area and centroid coordinate fields for later merges and plotting.

---

## Notes / QA checks recommended
- Verify **unique `CW_Id` counts** are consistent:
  - riparian-erased wetlands vs. pour points vs. dissolved watershed polygons.
- If counts differ, inspect:
  - wetlands disappearing due to shoreline/riparian erases,
  - pour points falling outside valid drainage (before snapping),
  - watersheds splitting (should be fixed by dissolving on `CW_Id`).



## 0) Requirements

- ArcGIS Pro / arcpy with Spatial Analyst.
- Flow direction raster (`D8_flow`) and **flow accumulation** raster (`FlowAcc`) on the same grid.
- Wetland-connected polygons for each scenario (avg/high/low/surge), each with a stable id field (we standardize to `CW_Id`).

**What is `avg_pourpoints.shp`?**  
It is a **point feature class** with **one point per wetland** (per `CW_Id`). These points are snapped to the highest flow-accumulation cell nearby, then used as pour points for the Watershed tool.


In [1]:
import os
import arcpy
from arcpy import env
from arcpy.sa import SnapPourPoint, Watershed
from arcpy import sa
import numpy as np
arcpy.env.overwriteOutput = True
arcpy.CheckOutExtension("Spatial")
arcpy.env.addOutputsToMap = False  # helps avoid schema locks

## 1) Inputs / outputs 

Fill in your real paths. Keep the same projected CRS as your DEM / flow rasters (often EPSG:3174/3175 for Great Lakes Albers).


In [2]:

# -------------------------------------------------------------------
# Inputs
# -------------------------------------------------------------------

inDir = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer"
inDCW = r"D:\Users\abolmaal\data\coastalwetlands\finalwetland"
CW_path = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Coastalwetland\hitshoreline"

wetlands_avg_inun_original   = os.path.join(inDCW, "wetlands_connected_avg_inundation_GLAlbers.shp")
wetlands_high_inun_original  = os.path.join(inDCW, "wetlands_connected_high_inundation_GLAlbers.shp")
wetlands_low_inun_original   = os.path.join(inDCW, "wetlands_connected_low_inundation_GLAlbers.shp")
wetlands_surge_original      = os.path.join(inDCW, "wetlands_connected_surge_inundation_GLAlbers.shp")

inStreams = os.path.join(inDir, "GLB_Stream", "GLB_stream_Ras_FeatureToLine.shp")
D8_flow   = r"S:\Projects\Active\GLB_Nutrient_Transport\DEM_rasters\GLB_Bdry_buff10km_dem_fill_dir.tif"
flowacc = r"S:\Projects\Active\GLB_Nutrient_Transport\DEM_rasters\GLB_Bdry_buff10km_dem_fill_flowaccu.tif"
inStreamsWatershed = os.path.join(inDir, "Streamwatershed", "PointWaterdhed_LH.shp")

Lake_Huron = r"D:\Users\abolmaal\code\boundry\hydro_p_LakeHuron\hydro_p_LakeHuron.shp"


shoreline_fvcome = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Basins\FVCOME\FVCOM_shoreline.shp"

# -------------------------------------------------------------------
# Parameters / field names
# -------------------------------------------------------------------
CW_ID_FIELD = "CW_Id"          # stable wetland id
COASTAL_ID_FIELD = "Coastal_Id" # optional (we'll set equal to CW_Id unless you want different)
COASTAL_Area_FIELD = "CW_Aream2"  # area of coastal watershed in m2
Wetland_Area_FIELD = "WS_Aream2"  # area of wetland in m2
crs_Albers = arcpy.SpatialReference(3174)  # Great Lakes Albers meters
crs_WGS84  = arcpy.SpatialReference(4326)

# -------------------------------------------------------------------
# Outputs (your folders)
# -------------------------------------------------------------------
outDir_stream = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\CoastalWatersheds\GLB_Stream"
outBuffer = os.path.join(outDir_stream, "GLB_stream_Ras_FeatureToLine_50m.shp")

outpath = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\CoastalWatersheds"
outErase_Riper   = os.path.join(outpath, "Erase_Riperian")
outErase_drainage= os.path.join(outpath, "Erase_drainage")
outErase_Lake    = os.path.join(outpath, "Erase_lake")
outPourpoints    = os.path.join(outpath, "Pourpoints")
outWatersheds    = os.path.join(outpath, "Watershed_rasters")

for d in [outDir_stream, outErase_Riper, outErase_drainage, outErase_Lake, outPourpoints, outWatersheds]:
    os.makedirs(d, exist_ok=True)

shorebuffer = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Basins\FVCOME\FVCOM_shoreline_2000buffer.shp"

wetlands_avg_inun  = os.path.join(CW_path, "Wetland_connected_avg_inundation_NAD1983_shorelineinteraction_buffer2000m.shp")
wetlands_low_inun  = os.path.join(CW_path, "Wetland_connected_low_inundation_NAD1983_shorelineinteraction_buffer2000m.shp")
wetlands_high_inun = os.path.join(CW_path, "Wetland_connected_high_inundation_NAD1983_shorelineinteraction_buffer2000m.shp")
wetlands_surge     = os.path.join(CW_path, "Wetland_connected_surge_inundation_NAD1983_shorelineinteraction_buffer2000m.shp")

erase_buffer_avg   = os.path.join(outErase_Riper, "Wetland_connected_avg_erasebuff_50.shp")
erase_buffer_high  = os.path.join(outErase_Riper, "Wetland_connected_high_erasebuff_50.shp")
erase_buffer_low   = os.path.join(outErase_Riper, "Wetland_connected_low_erasebuff_50.shp")
erase_buffer_surge = os.path.join(outErase_Riper, "Wetland_connected_surge_erasebuff_50.shp")

CoastalWatershed_avg_erase_lakedrain  = os.path.join(outErase_drainage, "CoastalWatershed_avg_erase_lakedrain.shp")
CoastalWatershed_high_erase_lakedrain = os.path.join(outErase_drainage, "CoastalWatershed_high_erase_lakedrain.shp")
CoastalWatershed_low_erase_lakedrain  = os.path.join(outErase_drainage, "CoastalWatershed_low_erase_lakedrain.shp")
CoastalWatershed_surge_erase_lakedrain= os.path.join(outErase_drainage, "CoastalWatershed_surge_erase_lakedrain.shp")

CoastalWatershed_avg_erase_lakedrain_LakeHuron   = os.path.join(outErase_Lake, "CoastalWatershed_avg_erase_lakedrain_LakeHuron.shp")
CoastalWatershed_high_erase_lakedrain_LakeHuron  = os.path.join(outErase_Lake, "CoastalWatershed_high_erase_lakedrain_LakeHuron.shp")
CoastalWatershed_low_erase_lakedrain_LakeHuron   = os.path.join(outErase_Lake, "CoastalWatershed_low_erase_lakedrain_LakeHuron.shp")
CoastalWatershed_surge_erase_lakedrain_LakeHuron = os.path.join(outErase_Lake, "CoastalWatershed_surge_erase_lakedrain_LakeHuron.shp")



## 2) Helper functions

These helpers:

- enforce a stable `CW_Id`
- create **one** pour point per `CW_Id`
- snap pour points to high flow accumulation
- run watershed (raster) and convert back to polygons while preserving ids
- compute areas + WGS84 centroid lat/lon
- run sanity checks for missing ids


In [3]:
# -------------------------------------------------------------------
# Helper functions
# -------------------------------------------------------------------
def ensure_field(fc, name, ftype="DOUBLE"):
    # Shapefile limit: 10 chars
    if arcpy.Describe(fc).dataType == "ShapeFile" and len(name) > 10:
        short = name[:10]
        print(f"⚠️ Shapefile field '{name}' too long -> using '{short}'")
        name = short

    fields = [f.name for f in arcpy.ListFields(fc)]
    if name not in fields:
        arcpy.management.AddField(fc, name, ftype)
    return name

def calculate_area_m2(fc, out_field):
    out_field = ensure_field(fc, out_field, "DOUBLE")
    arcpy.management.CalculateGeometryAttributes(
        fc, [[out_field, "AREA"]], area_unit="SQUARE_METERS"
    )
    return out_field

def add_xy_ll(fc, prefix, src_crs=crs_Albers):
    """
    Adds:
      {prefix}_cx, {prefix}_cy  (in src_crs units, meters for 3174)
      {prefix}_lon, {prefix}_lat (in WGS84 DD)
    """
    ensure_field(fc, f"{prefix}_cx", "DOUBLE")
    ensure_field(fc, f"{prefix}_cy", "DOUBLE")
    arcpy.management.CalculateField(fc, f"{prefix}_cx", "!SHAPE.centroid.X!", "PYTHON3")
    arcpy.management.CalculateField(fc, f"{prefix}_cy", "!SHAPE.centroid.Y!", "PYTHON3")

    ensure_field(fc, f"{prefix}_lon", "DOUBLE")
    ensure_field(fc, f"{prefix}_lat", "DOUBLE")
    # CalculateGeometryAttributes supports centroid in a specified coordinate system
    arcpy.management.CalculateGeometryAttributes(
        fc,
        [[f"{prefix}_lat", "CENTROID_Y"], [f"{prefix}_lon", "CENTROID_X"]],
        coordinate_system=crs_WGS84,
        coordinate_format="DD"
    )

def count_ids(fc, id_field):
    ids = set()
    with arcpy.da.SearchCursor(fc, [id_field]) as cur:
        for (v,) in cur:
            if v is not None:
                ids.add(int(v))
    return len(ids)

def make_pourpoints(wetlands_fc, out_points_fc, id_field=CW_ID_FIELD):
    """
    1) Dissolve by CW_Id -> single multipart per CW_Id
    2) FeatureToPoint INSIDE -> 1 point per CW_Id
    """
    tmp_diss = os.path.join("in_memory", "tmp_diss")
    if arcpy.Exists(tmp_diss):
        arcpy.management.Delete(tmp_diss)

    arcpy.management.Dissolve(wetlands_fc, tmp_diss, dissolve_field=id_field)
    arcpy.management.FeatureToPoint(tmp_diss, out_points_fc, "INSIDE")
    arcpy.management.Delete(tmp_diss)

def snap_pourpoints(in_points, flowacc_raster, out_points, snap_dist="200 Meters"):
    """
    SnapPourPoint expects a flow accumulation raster.
    """
    out_ras = os.path.join("in_memory", "snapped_pp_ras")
    if arcpy.Exists(out_ras):
        arcpy.management.Delete(out_ras)

    # SnapPourPoint returns a raster. We'll convert to points with value preserved.
    snapped = arcpy.sa.SnapPourPoint(in_points, flowacc_raster, snap_dist, CW_ID_FIELD)
    snapped.save(out_ras)

    # RasterToPoint creates points with "grid_code"
    arcpy.conversion.RasterToPoint(out_ras, out_points, "VALUE")

    # Move snapped raster value -> CW_Id
    ensure_field(out_points, CW_ID_FIELD, "LONG")
    arcpy.management.CalculateField(out_points, CW_ID_FIELD, "!grid_code!", "PYTHON3")
    arcpy.management.Delete(out_ras)

def watershed_from_points(flowdir, snapped_points, out_watershed_raster):
    """
    Watershed raster values will equal CW_Id (because we pass CW_Id field).
    """
    ws = arcpy.sa.Watershed(flowdir, snapped_points, CW_ID_FIELD)
    ws.save(out_watershed_raster)

def watershed_raster_to_polygon(ws_raster, out_poly, id_field=CW_ID_FIELD):
    """
    RasterToPolygon -> gridcode. Then set CW_Id=gridcode and dissolve by CW_Id.
    """
    tmp_poly = os.path.join("in_memory", "tmp_ws_poly")
    if arcpy.Exists(tmp_poly):
        arcpy.management.Delete(tmp_poly)

    arcpy.conversion.RasterToPolygon(ws_raster, tmp_poly, "NO_SIMPLIFY", "VALUE")

    # Make sure CW_Id exists and equals gridcode
    ensure_field(tmp_poly, id_field, "LONG")
    arcpy.management.CalculateField(tmp_poly, id_field, "!gridcode!", "PYTHON3")

    # Dissolve to one watershed polygon per CW_Id (removes splits)
    arcpy.management.Dissolve(tmp_poly, out_poly, dissolve_field=id_field)

    arcpy.management.Delete(tmp_poly)
    
def watershed_from_snapped_raster(flowdir, snapped_ras, out_watershed_raster):
    """
    snapped_ras is the output of SnapPourPoint (a raster with CW_Id values).
    Watershed output raster will keep those CW_Id values.
    """
    ws = arcpy.sa.Watershed(flowdir, snapped_ras)
    ws.save(out_watershed_raster)
    
    
# -------------------------------------------------------------------


def _ensure_field(fc, field_name, field_type="LONG"):
    fields = [f.name for f in arcpy.ListFields(fc)]
    if field_name not in fields:
        arcpy.management.AddField(fc, field_name, field_type)

def unique_snap_points_to_flowacc_cells(
    in_points_fc,
    id_field,
    flowacc_raster,
    snap_dist,
    out_points_fc,
    max_expand_steps=3,
    expand_factor=1.5,
    allow_nodata=False,
):
    """
    Create a snapped points FC where each input point is moved to a UNIQUE raster cell
    chosen as the highest flow accumulation cell within snap_dist (expanded if needed).
    This guarantees 1 unique snapped cell per input ID (unless there aren't enough cells).
    """

    arcpy.env.overwriteOutput = True

    r = arcpy.Raster(flowacc_raster)
    sr = r.spatialReference
    cellw = float(r.meanCellWidth)
    cellh = float(r.meanCellHeight)
    ext = r.extent
    xmin, ymin, xmax, ymax = ext.XMin, ext.YMin, ext.XMax, ext.YMax

    # Determine raster size in cells (approx, but enough for indexing)
    ncol = int(round((xmax - xmin) / cellw))
    nrow = int(round((ymax - ymin) / cellh))

    # Create output FC
    out_dir = os.path.dirname(out_points_fc)
    out_name = os.path.basename(out_points_fc)
    if arcpy.Exists(out_points_fc):
        arcpy.management.Delete(out_points_fc)

    arcpy.management.CreateFeatureclass(
        out_dir, out_name, "POINT", spatial_reference=sr
    )
    _ensure_field(out_points_fc, id_field, "LONG")

    # Build a quick index of all input points
    pts = []
    with arcpy.da.SearchCursor(in_points_fc, ["SHAPE@XY", id_field]) as cur:
        for (x, y), cid in cur:
            if cid is None:
                continue
            pts.append((float(x), float(y), int(cid)))

    used_cells = set()  # (row_top, col)

    def xy_to_rowcol_top(x, y):
        col = int((x - xmin) / cellw)
        row_top = int((ymax - y) / cellh)
        return row_top, col

    def rowcol_top_to_cellcenter(row_top, col):
        x = xmin + (col + 0.5) * cellw
        y = ymax - (row_top + 0.5) * cellh
        return x, y

    def window_to_numpy(row_top, col, rad_cells):
        # clamp window bounds in raster indices
        r0 = max(0, row_top - rad_cells)
        r1 = min(nrow - 1, row_top + rad_cells)
        c0 = max(0, col - rad_cells)
        c1 = min(ncol - 1, col + rad_cells)

        # lower-left corner of the window in map units
        x_ll = xmin + c0 * cellw
        y_ll = ymax - (r1 + 1) * cellh

        nrows = (r1 - r0 + 1)
        ncols = (c1 - c0 + 1)

        # IMPORTANT: integer rasters can't use NaN for nodata_to_value
        nodata_sentinel = -9999

        arr = arcpy.RasterToNumPyArray(
            r,
            lower_left_corner=arcpy.Point(x_ll, y_ll),
            ncols=ncols,
            nrows=nrows,
            nodata_to_value=nodata_sentinel
        )

        # convert to float and set sentinel to NaN
        arr = arr.astype("float64")
        arr[arr == nodata_sentinel] = np.nan

        return arr, (r0, r1, c0, c1)

    def pick_best_unused_cell(arr, bounds):
        r0, r1, c0, c1 = bounds
        nrows, ncols = arr.shape

        # Flatten and sort by flowacc descending (nan ignored)
        flat = arr.ravel()
        valid_idx = np.where(np.isfinite(flat))[0]
        if valid_idx.size == 0:
            return None

        order = valid_idx[np.argsort(flat[valid_idx])[::-1]]

        for k in order:
            i = k // ncols   # array row index (0=bottom)
            j = k % ncols

            # convert (i,j) -> global raster (row_top, col)
            row_top = r1 - i
            col = c0 + j

            if (row_top, col) in used_cells:
                continue

            # if you want to forbid snapping into nodata/invalid, array already NaN-handled
            return row_top, col, float(arr[i, j])

        return None

    missing = []

    with arcpy.da.InsertCursor(out_points_fc, ["SHAPE@XY", id_field]) as icur:
        for x, y, cid in pts:
            row_top, col = xy_to_rowcol_top(x, y)

            # start radius in cells
            base_rad = int(np.ceil(snap_dist / cellw))

            chosen = None
            rad = base_rad

            for step in range(max_expand_steps + 1):
                arr, bounds = window_to_numpy(row_top, col, rad)
                chosen = pick_best_unused_cell(arr, bounds)
                if chosen is not None:
                    break
                rad = int(np.ceil(rad * expand_factor))

            if chosen is None:
                missing.append(cid)
                continue

            rtop, c, val = chosen
            used_cells.add((rtop, c))

            sx, sy = rowcol_top_to_cellcenter(rtop, c)
            icur.insertRow(((sx, sy), cid))

    print(f"✅ unique snapped points created: {len(pts) - len(missing)} / {len(pts)}")
    if missing:
        print(f"⚠️ Could not snap {len(missing)} points (no valid unused cells found). Example IDs: {missing[:10]}")
    return out_points_fc


def remove_overlaps_by_priority(in_fc, id_field, out_fc):
    _safe_delete(out_fc)

    # copy first
    tmp = os.path.join(ws_gdb, "tmp_nooverlap_copy")
    _safe_delete(tmp)
    gp("Copy for no-overlap", arcpy.management.CopyFeatures, in_fc, tmp)

    # add area
    area_f = _ensure_field(tmp, "A_M2", "DOUBLE")
    arcpy.management.CalculateGeometryAttributes(tmp, [[area_f, "AREA"]], area_unit="SQUARE_METERS")

    # sort by area desc (biggest wins)
    sorted_fc = os.path.join(ws_gdb, "tmp_nooverlap_sorted")
    _safe_delete(sorted_fc)
    gp("Sort by area", arcpy.management.Sort, tmp, sorted_fc, [[area_f, "DESCENDING"]])

    # iterative erase
    kept = os.path.join(ws_gdb, "tmp_nooverlap_kept")
    _safe_delete(kept)
    gp("Create kept FC", arcpy.management.CopyFeatures, sorted_fc, kept)  # start with all, then we'll rebuild

    _safe_delete(kept)
    gp("Create empty kept FC", arcpy.management.CreateFeatureclass,
       ws_gdb, os.path.basename(kept), "POLYGON", sorted_fc, "DISABLED", "DISABLED", D8_SR)

    kept_id = _ensure_field(kept, id_field, "LONG")

    # process one by one
    lyr = "lyr_sorted"
    arcpy.management.MakeFeatureLayer(sorted_fc, lyr)

    oids = [r[0] for r in arcpy.da.SearchCursor(sorted_fc, [arcpy.Describe(sorted_fc).OIDFieldName])]
    for oid in oids:
        arcpy.management.SelectLayerByAttribute(lyr, "NEW_SELECTION", f"OBJECTID = {oid}")
        piece = os.path.join(ws_gdb, "tmp_piece")
        _safe_delete(piece)
        arcpy.management.CopyFeatures(lyr, piece)

        # erase with what we've already kept
        if int(arcpy.management.GetCount(kept)[0]) > 0:
            erased = os.path.join(ws_gdb, "tmp_piece_erased")
            _safe_delete(erased)
            arcpy.analysis.Erase(piece, kept, erased)
            _safe_delete(piece)
            piece = erased

        if int(arcpy.management.GetCount(piece)[0]) > 0:
            arcpy.management.Append(piece, kept, "NO_TEST")

        _safe_delete(piece)

    arcpy.management.Delete(lyr)

    # final dissolve back to CW_Id
    gp("Dissolve no-overlap result", arcpy.management.Dissolve, kept, out_fc, id_field)

    _safe_delete(tmp); _safe_delete(sorted_fc); _safe_delete(kept)
    return out_fc


# -------------------------------------------------------------------
# 0) Add stable CW_Id, Coastal wetland area to ORIGINAL wetlands (IMPORTANT FIX)
#    Use existing "Id" if present; else fallback to OBJECTID/FID.
# ------------------------------------------------------------------


In [6]:
# -------------------------------------------------------------------
# Parameters / field names
# -------------------------------------------------------------------
CW_ID_FIELD        = "CW_Id"            # stable wetland id
COASTAL_ID_FIELD   = "Coastal_Id"       # optional (set equal to CW_Id here)
COASTAL_AREA_FIELD = "CW_Aream2"  # (here: wetland polygon area in m²)

wetlands_fcs = [
    wetlands_avg_inun_original,
    wetlands_high_inun_original,
    wetlands_low_inun_original,
    wetlands_surge_original,
]

for fc in wetlands_fcs:

    # 1) Ensure fields exist (adds them if missing)
    ensure_field(fc, CW_ID_FIELD, "LONG")
    ensure_field(fc, COASTAL_ID_FIELD, "LONG")
    ensure_field(fc, COASTAL_AREA_FIELD, "DOUBLE")

    # 2) Re-read fields AFTER ensuring (important!)
    fields = [f.name for f in arcpy.ListFields(fc)]

    # 3) Populate CW_Id and Coastal_Id
    if "Id" in fields:
        arcpy.management.CalculateField(fc, CW_ID_FIELD, "!Id!", "PYTHON3")
        arcpy.management.CalculateField(fc, COASTAL_ID_FIELD, "!Id!", "PYTHON3")
    else:
        oid = arcpy.Describe(fc).OIDFieldName
        arcpy.management.CalculateField(fc, CW_ID_FIELD, f"!{oid}!", "PYTHON3")
        arcpy.management.CalculateField(fc, COASTAL_ID_FIELD, f"!{oid}!", "PYTHON3")

    # 4) Compute polygon area (m²) for THIS layer (wetland area)
    #    This uses the dataset CRS units. If fc is EPSG:3174, area will be m².
    try:
        arcpy.management.CalculateGeometryAttributes(
            fc,
            [[COASTAL_AREA_FIELD, "AREA"]],
            area_unit="SQUARE_METERS"
        )
    except Exception as e:
        print(f"⚠️ Could not calculate {COASTAL_AREA_FIELD} for {os.path.basename(fc)}: {e}")

    print(f"✅ ensured fields + IDs + area on: {os.path.basename(fc)}")



✅ ensured fields + IDs + area on: wetlands_connected_avg_inundation_GLAlbers.shp
✅ ensured fields + IDs + area on: wetlands_connected_high_inundation_GLAlbers.shp
✅ ensured fields + IDs + area on: wetlands_connected_low_inundation_GLAlbers.shp
✅ ensured fields + IDs + area on: wetlands_connected_surge_inundation_GLAlbers.shp



# -------------------------------------------------------------------
# 1) Shoreline buffer (2000m) in EPSG:3174
# -------------------------------------------------------------------

In [5]:
shoreline_3174 = os.path.join("in_memory", "shoreline_3174")
arcpy.management.Project(shoreline_fvcome, shoreline_3174, crs_Albers)

if not arcpy.Exists(shorebuffer):
    arcpy.analysis.Buffer(shoreline_3174, shorebuffer, "2000 Meters", dissolve_option="ALL")

arcpy.management.Delete(shoreline_3174)

<Result 'true'>

# -------------------------------------------------------------------
# 2) Intersect wetlands with shoreline buffer (keeps CW_Id)
# -------------------------------------------------------------------

In [6]:
# cw_pairs = [
#     (wetlands_avg_inun_original,  wetlands_avg_inun),
#     (wetlands_low_inun_original,  wetlands_low_inun),
#     (wetlands_high_inun_original, wetlands_high_inun),
#     (wetlands_surge_original,     wetlands_surge),
# ]

# for in_fc, out_fc in cw_pairs:
#     # project wetlands to 3174
#     tmp_3174 = os.path.join("in_memory", "cw_3174")
#     arcpy.management.Project(in_fc, tmp_3174, crs_Albers)

#     tmp_int = os.path.join("in_memory", "cw_int")
#     arcpy.analysis.Intersect([tmp_3174, shorebuffer], tmp_int, "ALL")

#     # Save in 3174 (recommended). If you really need original CRS, project back here.
#     arcpy.management.CopyFeatures(tmp_int, out_fc)

#     arcpy.management.Delete(tmp_3174)
#     arcpy.management.Delete(tmp_int)

#     print(f"✅ shoreline-intersect: {os.path.basename(out_fc)}")

# -------------------------------------------------------------------
# 2) Stream riparian buffer 50 m + erase wetlands (keeps CW_Id)
- Create a 50 meter buffer for Great lakes basin streams (This is riverin Riperian area)
-  Erase your coastal wetlands that overlap with Riperian area(GLB streams)
# -------------------------------------------------------------------

In [8]:
if not arcpy.Exists(outBuffer):
    arcpy.analysis.Buffer(inStreams, outBuffer, "50 Meters")

arcpy.analysis.Erase(wetlands_avg_inun_original, outBuffer, erase_buffer_avg)
arcpy.analysis.Erase(wetlands_high_inun_original, outBuffer, erase_buffer_high)
arcpy.analysis.Erase(wetlands_low_inun_original, outBuffer, erase_buffer_low)
arcpy.analysis.Erase(wetlands_surge_original, outBuffer, erase_buffer_surge)

#arcpy.management.ClearWorkspaceCache()
# add wetland areas and coordinates
# for fc in [erase_buffer_avg, erase_buffer_high, erase_buffer_low, erase_buffer_surge]:
#     # shorter name for SHP + avoids long names anyway
#     #calculate_area_m2(fc, "CW_Area_m2")     # if this is SHP it's OK (<10 chars)
#     add_xy_ll(fc, prefix="CW")
#     print(f"✅ wetland attrs added: {os.path.basename(fc)}")


<Result 'D:\\Users\\abolmaal\\Arcgis\\NASAOceanProject\\GIS_layer\\CoastalWatersheds\\Erase_Riperian\\Wetland_connected_surge_erasebuff_50.shp'>

In [4]:
# Add CW_cx and CW_cy to the erase_buffer_avg
arcpy.env.overwriteOutput = True

# ---- inputs (your shapefiles) ----
shps = [
    erase_buffer_avg,
    erase_buffer_high,
    erase_buffer_low,
    erase_buffer_surge
]

TARGET_CRS = arcpy.SpatialReference(3174)  # NAD_1983_Great_Lakes_Basin_Albers

def ensure_field(fc, name, ftype="DOUBLE"):
    existing = [f.name for f in arcpy.ListFields(fc)]
    if name not in existing:
        arcpy.management.AddField(fc, name, ftype)
        print(f"✅ Added field {name} to {os.path.basename(fc)}")

def add_centroid_and_area_3174(fc):
    # Ensure fields exist
    ensure_field(fc, "CW_cx", "DOUBLE")
    ensure_field(fc, "CW_cy", "DOUBLE")
    ensure_field(fc, "CW_Aream2", "DOUBLE")

    # If fc not in 3174, project geometry on-the-fly for centroid/area calculations
    sr = arcpy.Describe(fc).spatialReference
    needs_proj = (sr is None) or (sr.factoryCode != 3174)

    fields = ["SHAPE@", "CW_cx", "CW_cy", "CW_Aream2"]
    with arcpy.da.UpdateCursor(fc, fields) as cur:
        for shp, cx, cy, area in cur:
            if shp is None:
                continue

            geom = shp.projectAs(TARGET_CRS) if needs_proj else shp

            c = geom.centroid
            cx_new, cy_new = c.X, c.Y
            area_new = geom.area  # m² in 3174

            cur.updateRow((shp, cx_new, cy_new, area_new))

    print(f"✅ Calculated CW_cx/CW_cy/CW_Aream2 for: {os.path.basename(fc)}")

# ---- run on all four ----
for fc in shps:
    add_centroid_and_area_3174(fc)

✅ Added field CW_cx to Wetland_connected_avg_erasebuff_50.shp
✅ Added field CW_cy to Wetland_connected_avg_erasebuff_50.shp
✅ Calculated CW_cx/CW_cy/CW_Aream2 for: Wetland_connected_avg_erasebuff_50.shp
✅ Added field CW_cx to Wetland_connected_high_erasebuff_50.shp
✅ Added field CW_cy to Wetland_connected_high_erasebuff_50.shp
✅ Calculated CW_cx/CW_cy/CW_Aream2 for: Wetland_connected_high_erasebuff_50.shp
✅ Added field CW_cx to Wetland_connected_low_erasebuff_50.shp
✅ Added field CW_cy to Wetland_connected_low_erasebuff_50.shp
✅ Calculated CW_cx/CW_cy/CW_Aream2 for: Wetland_connected_low_erasebuff_50.shp
✅ Added field CW_cx to Wetland_connected_surge_erasebuff_50.shp
✅ Added field CW_cy to Wetland_connected_surge_erasebuff_50.shp
✅ Calculated CW_cx/CW_cy/CW_Aream2 for: Wetland_connected_surge_erasebuff_50.shp


## Step 4 — Create **1:1 Coastal-Wetland Watersheds** (Pourpoints → Snap → Watershed → Polygons → **Clip overlaps** → **Repair missing IDs**)

This cell delineates **one coastal watershed per coastal wetland (`CW_Id`)** for each inundation category (**avg/high/low/surge**), while ensuring the identifier **`CW_Id` remains 1:1** throughout the workflow.

Unlike earlier versions that could *drop entire IDs* when wetlands were in-lake or when watersheds overlapped stream-drainage areas, this updated workflow:

* **keeps every `CW_Id`**, even if the wetland is partially/fully in the lake,
* **clips out only the overlapping portions** (lake interior and stream-watershed overlap),
* and **repairs** missing IDs caused by raster snapping collisions by rebuilding only the missing IDs one-by-one.

It is also designed to avoid common ArcPy issues (schema locks, field-name limits in shapefiles, field-case mismatches, and SnapPourPoint ID collapsing).

---

### What this cell produces (per inundation category: avg/high/low/surge)

For each category, the cell creates:

* **Pourpoints (GDB feature class)**: one point per `CW_Id` (land-only + fallback for lake-only IDs)
* **Snapped pourpoints (GDB feature class)**: snapped onto high-flowacc land cells (using `flowacc_land`)
* **Watershed raster (GDB raster)**: watershed labels equal to `CW_Id`
* **Watershed polygons (GDB feature class)**: raster converted to polygons and dissolved to **one polygon per `CW_Id`**
* **Clipped watershed polygons (final)**: only the portions overlapping:

  * **Lake Huron interior**, and
  * **stream-watershed mask**
    are removed (the rest of the polygon is preserved)
* **Repaired final outputs (shapefiles)**:

  * `*_erase_lakedrain_LakeHuron.shp` (one feature per `CW_Id`, after clip + repair)
* **Final attributes added to the final shapefile** (when possible):

  * `WS_AREAM2` = watershed area (m²)
  * `WS_cx`, `WS_cy` = centroid X/Y in dataset CRS
  * `WS_lon`, `WS_lat` = centroid lon/lat in WGS84

---

### Why “pourpoints” matter

A **pour point** is the location Spatial Analyst uses to define **which upstream cells contribute** to that outlet.
Here, each wetland gets **exactly one pourpoint per `CW_Id`**, which drives the “one watershed per wetland” requirement.

---

### Updated snapping logic (and why it differs from the older “unique snap” approach)

Previously, strict uniqueness snapping (one raster cell per point) was used to prevent ID collisions. In practice, nearshore lake-only wetlands can still collide at shoreline cells and/or fail to reach land.

This updated workflow uses a **two-stage strategy**:

1. **Bulk snapping** (fast): uses SnapPourPoint on a land-only flowacc surface (`flowacc_land`)
2. **Repair pass** (precise): if any `CW_Id` is missing after delineation + clipping, rebuild just those IDs **one-by-one** to eliminate raster collisions.

This preserves performance (bulk) while guaranteeing completeness (repair).

---

### Core processing steps inside the loop (per category)

#### 0) Environment + workspaces (no C:\ temp writes)

* All intermediates are written to:

  * `watersheds.gdb` (under `outWatersheds`)
  * `pourpoints.gdb` (under `outPourpoints`)
* Processing is aligned to the D8 grid:

  * `snapRaster = D8_flow`
  * `extent = D8_flow`
  * `cellSize = D8_flow`
  * `outputCoordinateSystem = D8_SR`

---

#### 1) Build masks once (outside the loop)

* **Stream-watershed mask**

  * CRS fixed if mislabeled, geometry repaired, dissolved, and (optionally) buffered slightly
* **Lake Huron polygon**

  * CRS corrected and projected to D8 CRS
* **Land-only flowacc surface**

  * `flowacc_land = flowacc` with lake cells set to NoData
  * ensures snapping targets land cells only

---

#### 2) Prepare wetlands and authoritative CW_Id list

* Wetlands are projected to D8 CRS and repaired
* **Original wetlands dissolved by `CW_Id`** to guarantee:

  * one feature per ID
  * a definitive list of all IDs that must exist in the final output

✅ Output: `{cat}_wet_orig_diss` (GDB)

---

#### 3) Create pourpoints (one per CW_Id, including lake-only)

* Create **land-only wetlands** by erasing the lake, then dissolve by `CW_Id`
* Create pourpoints inside land-only dissolved polygons (1 per CW_Id)
* Identify **lake-only IDs** (present in original dissolve but missing from land-only dissolve)
* Create fallback pourpoints for lake-only IDs using the **original dissolved** polygons
* Append fallback points into the pourpoint set

✅ Output: `{cat}_pp_inside` (GDB; 1 point per CW_Id target)

---

#### 4) Snap pourpoints to land-only high-flowacc cells (bulk)

* Convert pourpoints to raster (VALUE = `CW_Id`)
* Snap via:

  * `SnapPourPoint(pour_raster, flowacc_land, snap_dist_m)`
* Convert snapped raster back to points and confirm ID coverage

✅ Outputs:

* `{cat}_pp_snapped_pts`
* `{cat}_pp_snap_ras`

---

#### 5) Delineate watershed raster (VALUE = CW_Id)

* `Watershed(D8_flow, snapped_pourpoint_raster)`
* Produces a raster where each watershed is labeled by `CW_Id`

✅ Output: `{cat}_ws_ras`

---

#### 6) Raster → polygon + dissolve to one feature per CW_Id

* `RasterToPolygon` creates `GRIDCODE`
* Compute `CW_Id = GRIDCODE`
* Dissolve by `CW_Id` to enforce **one polygon per wetland**

✅ Output: `{cat}_ws_poly`

---

#### 7) Clip overlaps (remove only the overlapping parts)

To avoid dropping whole watersheds, lake/stream constraints are applied as **polygon clip operations**, not as raster masks:

* Erase **Lake Huron** interior from watershed polygons
* Erase **stream-watershed mask** overlap from watershed polygons
* Dissolve again by `CW_Id`

✅ Output: final category shapefile `CoastalWatershed_{cat}_erase_lakedrain_LakeHuron.shp`

---

#### 8) Repair missing IDs (guarantee 1:1 output)

After clipping, some IDs may still be missing due to:

* SnapPourPoint raster collisions (multiple IDs snapped to same cell), or
* the clipped result becoming empty for a rare ID

Repair strategy:

* Compare final `CW_Id`s to the authoritative list (`wet_orig_diss`)
* For each missing `CW_Id`:

  * rebuild watershed **one-by-one** from its pourpoint (avoids collisions),
  * clip lake/stream overlaps,
  * append to final output,
  * dissolve to enforce one feature per `CW_Id`

This step ensures the final output is as close as possible to **1 polygon per wetland ID** while still honoring clipping rules.

---

### Built-in sanity checks (what to watch in the console)

The cell prints (per category):

* `unique wetland IDs (original, dissolved)`
* `unique snapped pourpoint IDs (target wet_n)`
* `watersheds BEFORE clipping unique IDs`
* `final watersheds AFTER clip unique IDs`
* `Missing IDs after clip`
* `final watersheds AFTER rebuild`

---

### Common adjustments you may want

* If many IDs are missing after clip:

  * increase `snap_dist_m` (lake-only points may need more distance to reach land)
  * reduce stream buffer distance if it removes too much nearshore area
* If you get schema lock issues:

  * close attribute tables, stop drawing layers, avoid adding outputs to map during run
* If the repair step runs long:

  * it scales with the number of missing IDs; reducing collision likelihood (snap distance + point density) reduces repair workload

---


In [ ]:
# # --- FINAL ROBUST CELL (UPDATED): 1 Coastal Watershed per Coastal Wetland (CW_Id)
# # Goal:
# #   - ALWAYS produce one watershed per CW_Id
# #   - Then CLIP OUT only the portions inside Lake Huron or inside Stream-Watershed mask
# #   - Do NOT drop whole IDs due to tiny overlaps
# #   - No C:\ temp writes (all temp in ws_gdb / pp_gdb)
# # Notes:
# #   - This workflow can still lose some CW_Ids due to SnapPourPoint raster collisions.
# #     We fix that with a "REPAIR missing IDs" step that rebuilds only missing CW_Ids one-by-one.

# import os, gc, time, sys
# import arcpy
# from arcpy import sa

# arcpy.env.overwriteOutput = True
# arcpy.env.addOutputsToMap = False
# arcpy.CheckOutExtension("Spatial")

# # -------------------------
# # REQUIRED: folders exist (must be defined in your notebook)
# # -------------------------
# # outPourpoints, outWatersheds, inStreamsWatershed, Lake_Huron, D8_flow
# # erase_buffer_avg/high/low/surge
# # CoastalWatershed_* output paths must exist in your notebook variables
# os.makedirs(outPourpoints, exist_ok=True)
# os.makedirs(outWatersheds, exist_ok=True)

# # -------------------------
# # Reliable workspaces (GDBs)
# # -------------------------
# pp_gdb = os.path.join(outPourpoints, "pourpoints.gdb")
# if not arcpy.Exists(pp_gdb):
#     arcpy.management.CreateFileGDB(outPourpoints, "pourpoints.gdb")

# ws_gdb = os.path.join(outWatersheds, "watersheds.gdb")
# if not arcpy.Exists(ws_gdb):
#     arcpy.management.CreateFileGDB(outWatersheds, "watersheds.gdb")

# # FORCE all scratch/workspace to your ws_gdb (no C:\temp)
# arcpy.env.workspace = ws_gdb
# arcpy.env.scratchWorkspace = ws_gdb

# # -------------------------
# # Inputs
# # -------------------------
# flowacc = r"S:\Projects\Active\GLB_Nutrient_Transport\DEM_rasters\GLB_Bdry_buff10km_dem_fill_flowaccu.tif"
# print(f"✅ flowacc: {flowacc}", flush=True)

# cats = {
#     #"avg":   (erase_buffer_avg,   CoastalWatershed_avg_erase_lakedrain,   CoastalWatershed_avg_erase_lakedrain_LakeHuron),
#     "high":  (erase_buffer_high,  CoastalWatershed_high_erase_lakedrain,  CoastalWatershed_high_erase_lakedrain_LakeHuron),
#     "low":   (erase_buffer_low,   CoastalWatershed_low_erase_lakedrain,   CoastalWatershed_low_erase_lakedrain_LakeHuron),
#     "surge": (erase_buffer_surge, CoastalWatershed_surge_erase_lakedrain, CoastalWatershed_surge_erase_lakedrain_LakeHuron),
# }

# CW_ID_FIELD = "CW_Id"

# # -------------------------
# # Align env to D8 grid
# # -------------------------
# D8_SR = arcpy.Describe(D8_flow).spatialReference
# arcpy.env.snapRaster = D8_flow
# arcpy.env.cellSize   = D8_flow
# arcpy.env.extent     = D8_flow
# arcpy.env.outputCoordinateSystem = D8_SR
# cellsize = float(arcpy.Describe(D8_flow).meanCellWidth)

# print(f"✅ workspace: {ws_gdb}", flush=True)
# print(f"✅ scratch:   {ws_gdb}", flush=True)
# print(f"✅ D8_flow CRS: {D8_SR.name} (factoryCode={D8_SR.factoryCode})", flush=True)

# # ============================================================
# # Helpers
# # ============================================================
# def _log(msg):
#     print(msg, flush=True)
#     sys.stdout.flush()

# def _clear_locks():
#     try:
#         arcpy.ClearWorkspaceCache_management()
#     except Exception:
#         pass
#     gc.collect()

# def _safe_delete(p):
#     try:
#         if arcpy.Exists(p):
#             arcpy.management.Delete(p)
#     except Exception:
#         _clear_locks()
#         if arcpy.Exists(p):
#             arcpy.management.Delete(p)

# def _field_map_lower(fc):
#     return {f.name.lower(): f.name for f in arcpy.ListFields(fc)}

# def _find_field(fc, candidates):
#     fmap = _field_map_lower(fc)
#     for c in candidates:
#         if c and c.lower() in fmap:
#             return fmap[c.lower()]
#     return None

# def get_field_name_ci(fc, target_name):
#     if not target_name:
#         return None
#     t = target_name.lower()
#     for f in arcpy.ListFields(fc):
#         if f.name.lower() == t:
#             return f.name
#     return None

# def _is_shp(path):
#     return isinstance(path, str) and path.lower().endswith(".shp")

# def _ensure_field(fc, desired_name, field_type="LONG", fallbacks=()):
#     if not desired_name or not str(desired_name).strip():
#         desired_name = "CW_Id"

#     existing = get_field_name_ci(fc, desired_name)
#     if existing:
#         return existing

#     candidates = [desired_name] + list(fallbacks)
#     for nm in candidates:
#         if not nm:
#             continue

#         safe = arcpy.ValidateFieldName(nm, os.path.dirname(fc) if isinstance(fc, str) else "")
#         if _is_shp(fc) and len(safe) > 10:
#             safe = safe[:10]

#         existing = get_field_name_ci(fc, safe)
#         if existing:
#             return existing

#         try:
#             arcpy.management.AddField(fc, safe, field_type)
#             return get_field_name_ci(fc, safe) or safe
#         except Exception:
#             _clear_locks()
#             continue

#     raise RuntimeError(f"Cannot add field '{desired_name}' to {fc}")

# def count_unique(fc, id_field):
#     fld = get_field_name_ci(fc, id_field) or _find_field(fc, [id_field])
#     if not fld:
#         return 0
#     vals = set()
#     with arcpy.da.SearchCursor(fc, [fld]) as cur:
#         for (v,) in cur:
#             if v is not None:
#                 vals.add(int(v))
#     return len(vals)

# def _idset(fc, id_field):
#     fld = get_field_name_ci(fc, id_field) or _find_field(fc, [id_field])
#     s = set()
#     with arcpy.da.SearchCursor(fc, [fld]) as cur:
#         for (v,) in cur:
#             if v is not None:
#                 s.add(int(v))
#     return s

# def calculate_area_m2(fc, field="WS_AREAM2"):
#     try:
#         field = _ensure_field(fc, field, "DOUBLE", fallbacks=("AREA_M2","A_M2","AREA"))
#         arcpy.management.CalculateGeometryAttributes(fc, [[field, "AREA"]], area_unit="SQUARE_METERS")
#     except Exception as e:
#         _log(f"⚠️ area field skipped: {e}")
#     return field

# def add_xy_ll(fc, prefix="WS"):
#     try:
#         cx = _ensure_field(fc, f"{prefix}_cx", "DOUBLE", fallbacks=(f"{prefix}X",))
#         cy = _ensure_field(fc, f"{prefix}_cy", "DOUBLE", fallbacks=(f"{prefix}Y",))
#         arcpy.management.CalculateField(fc, cx, "!SHAPE.centroid.X!", "PYTHON3")
#         arcpy.management.CalculateField(fc, cy, "!SHAPE.centroid.Y!", "PYTHON3")

#         lon = _ensure_field(fc, f"{prefix}_lon", "DOUBLE", fallbacks=(f"{prefix}LON",))
#         lat = _ensure_field(fc, f"{prefix}_lat", "DOUBLE", fallbacks=(f"{prefix}LAT",))
#         arcpy.management.CalculateGeometryAttributes(
#             fc,
#             [[lat, "CENTROID_Y"], [lon, "CENTROID_X"]],
#             coordinate_system=arcpy.SpatialReference(4326),
#             coordinate_format="DD"
#         )
#     except Exception as e:
#         _log(f"⚠️ xy/ll fields skipped: {e}")

# def gp(label, func, *args, **kwargs):
#     _log(f"▶ {label}")
#     t0 = time.time()
#     out = func(*args, **kwargs)
#     _log(f"✅ DONE {label} ({(time.time()-t0)/60:.2f} min)")
#     return out

# # ---------- CRS FIX + PROJECT (TEMP WRITES INTO ws_gdb) ----------
# def fix_define_and_project_to_gdb(in_fc, out_fc, out_sr, assumed_src_if_mislabeled=None, name="layer"):
#     """
#     Robust CRS fixer:
#     - If dataset is mislabeled but coordinates already match out_sr, we:
#         CopyFeatures -> DefineProjection(out_sr) and STOP (no Project).
#     - Otherwise we DefineProjection to known source CRS (e.g. EPSG:4326) then Project.
#     All temp writes stay in ws_gdb (no C:\).
#     """
#     def _looks_like_degrees(ext):
#         return (abs(ext.XMin) <= 180 and abs(ext.XMax) <= 180 and abs(ext.YMin) <= 90 and abs(ext.YMax) <= 90)

#     def _looks_like_projected(ext):
#         return (max(abs(ext.XMin), abs(ext.XMax), abs(ext.YMin), abs(ext.YMax)) > 1000)

#     def _pick_transform(in_sr, out_sr):
#         try:
#             tx = arcpy.ListTransformations(in_sr, out_sr)
#             return tx[0] if tx else None
#         except Exception:
#             return None

#     _safe_delete(out_fc)

#     d = arcpy.Describe(in_fc)
#     sr = d.spatialReference
#     ext = d.extent

#     _log(f"\n[{name}] input: {in_fc}")
#     _log(f"[{name}] sr: {sr.name if sr else None} | factoryCode={getattr(sr,'factoryCode',None)} | type={getattr(sr,'type',None)}")
#     _log(f"[{name}] extent: XMin={ext.XMin:.3f} XMax={ext.XMax:.3f} YMin={ext.YMin:.3f} YMax={ext.YMax:.3f}")

#     tmp_copy = os.path.join(ws_gdb, f"tmp_{name}_copy")
#     _safe_delete(tmp_copy)

#     # Copy without any implicit projection
#     with arcpy.EnvManager(outputCoordinateSystem=None, extent=None, snapRaster=None, cellSize=None):
#         gp(f"CopyFeatures {name}", arcpy.management.CopyFeatures, in_fc, tmp_copy)

#     d2 = arcpy.Describe(tmp_copy)
#     sr2 = d2.spatialReference
#     ext2 = d2.extent

#     mislabeled_projected = (
#         sr2 is not None and getattr(sr2, "type", None) == "Geographic"
#         and _looks_like_projected(ext2) and not _looks_like_degrees(ext2)
#     )

#     if mislabeled_projected:
#         _log(f"[{name}] ⚠️ MISLABELED Geographic but coords are projected. SKIP Project; DefineProjection -> {out_sr.name}")
#         gp(f"DefineProjection {name}", arcpy.management.DefineProjection, tmp_copy, out_sr)
#         gp(f"Copy to output {name}", arcpy.management.CopyFeatures, tmp_copy, out_fc)
#         _safe_delete(tmp_copy)
#         return out_fc

#     generic_degrees = (
#         sr2 is not None and getattr(sr2, "type", None) == "Geographic"
#         and _looks_like_degrees(ext2)
#         and getattr(sr2, "factoryCode", 0) in (0, None)
#     )

#     if generic_degrees:
#         if assumed_src_if_mislabeled is None:
#             assumed_src_if_mislabeled = arcpy.SpatialReference(4326)
#         _log(f"[{name}] ⚠️ GENERIC geographic degrees. DefineProjection -> EPSG:4326 then Project.")
#         gp(f"DefineProjection {name}", arcpy.management.DefineProjection, tmp_copy, assumed_src_if_mislabeled)

#     sr_fixed = arcpy.Describe(tmp_copy).spatialReference
#     if sr_fixed and (sr_fixed.factoryCode == out_sr.factoryCode) and (sr_fixed.name == out_sr.name):
#         _log(f"[{name}] Already in target CRS. CopyFeatures only (no Project).")
#         gp(f"Copy to output {name}", arcpy.management.CopyFeatures, tmp_copy, out_fc)
#         _safe_delete(tmp_copy)
#         return out_fc

#     transform = _pick_transform(sr_fixed, out_sr)
#     _log(f"[{name}] Project -> {out_sr.name} | transform={transform}")

#     if transform:
#         gp(f"Project {name}", arcpy.management.Project, tmp_copy, out_fc, out_sr, transform)
#     else:
#         gp(f"Project {name}", arcpy.management.Project, tmp_copy, out_fc, out_sr)

#     _safe_delete(tmp_copy)
#     return out_fc

# def polygon_to_mask_raster(poly_fc, out_ras, value=1):
#     """
#     Polygon -> raster aligned to D8 (snap/cellsize/extent already set).
#     Uses a constant field to burn 'value' into raster.
#     """
#     _safe_delete(out_ras)
#     fld = "MASKVAL"
#     if fld not in [f.name for f in arcpy.ListFields(poly_fc)]:
#         arcpy.management.AddField(poly_fc, fld, "SHORT")
#         arcpy.management.CalculateField(poly_fc, fld, value, "PYTHON3")
#     gp(f"PolygonToRaster {os.path.basename(out_ras)}", arcpy.conversion.PolygonToRaster,
#        poly_fc, fld, out_ras, "CELL_CENTER", "", cellsize)
#     return out_ras

# # ============================================================
# # 0) Build projected masks ONCE (stored in ws_gdb)
# # ============================================================
# # Stream watershed mask: mislabeled as 4326 but projected coords
# inStreamsWS_tgt = os.path.join(ws_gdb, "inStreamsWS_tgt")
# fix_define_and_project_to_gdb(
#     in_fc=inStreamsWatershed,
#     out_fc=inStreamsWS_tgt,
#     out_sr=D8_SR,
#     name="inStreamsWS"
# )

# gp("RepairGeometry stream mask", arcpy.management.RepairGeometry, inStreamsWS_tgt)

# inStreams_single = os.path.join(ws_gdb, "inStreamsWS_single")
# _safe_delete(inStreams_single)
# gp("MultipartToSinglepart stream mask", arcpy.management.MultipartToSinglepart, inStreamsWS_tgt, inStreams_single)

# inStreamsWS_tgt_diss = os.path.join(ws_gdb, "inStreamsWS_tgt_diss")
# _safe_delete(inStreamsWS_tgt_diss)
# gp("Dissolve stream mask", arcpy.management.Dissolve, inStreams_single, inStreamsWS_tgt_diss)

# # Buffer stream mask (helps remove slivers; does NOT delete entire IDs because we clip polygons later)
# stream_buf_m = 60
# inStreamsWS_buf = os.path.join(ws_gdb, f"inStreamsWS_buf{stream_buf_m}m")
# _safe_delete(inStreamsWS_buf)
# gp("Buffer stream mask", arcpy.analysis.Buffer, inStreamsWS_tgt_diss, inStreamsWS_buf, f"{stream_buf_m} Meters", "FULL", "ROUND", "ALL")

# # Lake polygon: generic geographic in degrees
# Lake_tgt = os.path.join(ws_gdb, "LakeHuron_tgt")
# fix_define_and_project_to_gdb(
#     in_fc=Lake_Huron,
#     out_fc=Lake_tgt,
#     out_sr=D8_SR,
#     assumed_src_if_mislabeled=arcpy.SpatialReference(4326),
#     name="LakeHuron"
# )

# gp("RepairGeometry lake", arcpy.management.RepairGeometry, Lake_tgt)

# # Lake mask raster (for flowacc_land only)
# lake_mask_ras = os.path.join(ws_gdb, "LakeHuron_mask_ras")
# polygon_to_mask_raster(Lake_tgt, lake_mask_ras, value=1)
# _log(f"✅ lake_mask_ras: {lake_mask_ras}")

# # Land-only flowacc for snapping (NoData on lake)
# flowacc_land = os.path.join(ws_gdb, "flowacc_land")
# _safe_delete(flowacc_land)
# _log("▶ Build flowacc_land = flowacc where NOT lake")
# fa_land = sa.SetNull(sa.Raster(lake_mask_ras), sa.Raster(flowacc))
# fa_land.save(flowacc_land)
# _log(f"✅ flowacc_land: {flowacc_land}")

# # ============================================================
# # MAIN LOOP
# # ============================================================
# for cat, (wet_fc, out_ws_drain, out_ws_lake) in cats.items():

#     _log(f"\n==================== {cat.upper()} ====================")

#     # --- 1) Project wetlands into D8 SR (work in GDB)
#     wet_tgt = os.path.join(ws_gdb, f"{cat}_wet_tgt")
#     _safe_delete(wet_tgt)
#     gp(f"[{cat}] Project wetlands", arcpy.management.Project, wet_fc, wet_tgt, D8_SR)

#     wet_id_f = _ensure_field(wet_tgt, CW_ID_FIELD, "LONG", fallbacks=("CWID","CW_ID","CW_Id"))
#     gp(f"[{cat}] RepairGeometry wetlands", arcpy.management.RepairGeometry, wet_tgt)

#     # --- 2) Dissolve ORIGINAL wetlands by CW_Id (guarantees 1 polygon per CW_Id)
#     wet_orig_diss = os.path.join(ws_gdb, f"{cat}_wet_orig_diss")
#     _safe_delete(wet_orig_diss)
#     gp(f"[{cat}] Dissolve ORIGINAL wetlands by CW_Id", arcpy.management.Dissolve, wet_tgt, wet_orig_diss, wet_id_f)

#     wet_n = count_unique(wet_orig_diss, wet_id_f)
#     _log(f"[{cat}] unique wetland IDs (original, dissolved): {wet_n}")

#     # --- 3) LAND-ONLY wetlands (erase lake) then dissolve (for land pourpoints)
#     wet_land = os.path.join(ws_gdb, f"{cat}_wet_land")
#     _safe_delete(wet_land)
#     gp(f"[{cat}] Erase lake from wetlands (land-only)", arcpy.analysis.Erase, wet_tgt, Lake_tgt, wet_land)

#     wet_land_diss = os.path.join(ws_gdb, f"{cat}_wet_land_diss")
#     _safe_delete(wet_land_diss)
#     gp(f"[{cat}] Dissolve land-only wetlands by CW_Id", arcpy.management.Dissolve, wet_land, wet_land_diss, wet_id_f)

#     land_n = count_unique(wet_land_diss, wet_id_f)
#     _log(f"[{cat}] unique wetland IDs (land-only, dissolved): {land_n}")

#     # --- 4) Pourpoints from land-only dissolved (1 per CW_Id)
#     pp_inside = os.path.join(pp_gdb, f"{cat}_pp_inside")
#     _safe_delete(pp_inside)
#     gp(f"[{cat}] FeatureToPoint INSIDE (land-only dissolved)", arcpy.management.FeatureToPoint, wet_land_diss, pp_inside, "INSIDE")

#     # --- 5) Add fallback pourpoints for lake-only IDs from ORIGINAL dissolved (still 1 per CW_Id)
#     orig_ids = _idset(wet_orig_diss, wet_id_f)
#     land_ids = _idset(wet_land_diss, wet_id_f)
#     missing_lakeonly = sorted(list(orig_ids - land_ids))

#     if missing_lakeonly:
#         _log(f"⚠️ [{cat}] {len(missing_lakeonly)} wetlands appear fully in-lake after erase; adding 1 fallback point per CW_Id.")

#         # Select missing polygons from wet_orig_diss using a chunked IN() approach
#         wet_missing_poly = os.path.join(ws_gdb, f"{cat}_wet_missing_poly")
#         _safe_delete(wet_missing_poly)

#         lyr = f"lyr_{cat}_missing"
#         arcpy.management.MakeFeatureLayer(wet_orig_diss, lyr)
#         chunk = 900
#         for i in range(0, len(missing_lakeonly), chunk):
#             sub = missing_lakeonly[i:i+chunk]
#             where = f"{arcpy.AddFieldDelimiters(lyr, wet_id_f)} IN ({','.join(map(str, sub))})"
#             arcpy.management.SelectLayerByAttribute(lyr, "ADD_TO_SELECTION", where)
#         gp(f"[{cat} missing] Copy selected", arcpy.management.CopyFeatures, lyr, wet_missing_poly)
#         arcpy.management.Delete(lyr)

#         pp_fallback = os.path.join(pp_gdb, f"{cat}_pp_fallback")
#         _safe_delete(pp_fallback)
#         gp(f"[{cat}] FeatureToPoint INSIDE (fallback dissolved)", arcpy.management.FeatureToPoint, wet_missing_poly, pp_fallback, "INSIDE")

#         gp(f"[{cat}] Append fallback points", arcpy.management.Append, pp_fallback, pp_inside, "NO_TEST")
#         _safe_delete(pp_fallback)
#         _safe_delete(wet_missing_poly)

#     # ensure ID field exists on points
#     pp_id_f = _ensure_field(pp_inside, wet_id_f, "LONG", fallbacks=("CWID","CW_ID","CW_Id"))

#     # --- 6) SnapPourPoint in raster space (bulk) using LAND-ONLY flowacc
#     pp_ras = os.path.join(pp_gdb, f"{cat}_pp_ras")
#     _safe_delete(pp_ras)
#     gp(f"[{cat}] PointToRaster pourpoints", arcpy.conversion.PointToRaster,
#        pp_inside, pp_id_f, pp_ras, "MAXIMUM", "", cellsize)

#     snapped_pp_ras = os.path.join(pp_gdb, f"{cat}_pp_snapped_ras")
#     _safe_delete(snapped_pp_ras)

#     # For lake-only points you typically need a larger snap distance to reach land.
#     # But larger distances increase collisions. We'll keep it moderate and REPAIR missing IDs later.
#     snap_dist_m = 150
#     _log(f"[{cat}] SnapPourPoint distance = {snap_dist_m} m (land-only)")
#     sa.SnapPourPoint(sa.Raster(pp_ras), sa.Raster(flowacc_land), snap_dist_m).save(snapped_pp_ras)

#     # Convert snapped raster to points (bulk snapped pourpoints)
#     snapped_pts = os.path.join(pp_gdb, f"{cat}_pp_snapped_pts")
#     _safe_delete(snapped_pts)
#     gp(f"[{cat}] RasterToPoint snapped pourpoints", arcpy.conversion.RasterToPoint, snapped_pp_ras, snapped_pts, "VALUE")

#     val_field = _find_field(snapped_pts, ["GRID_CODE", "GRIDCODE", "VALUE"])
#     if not val_field:
#         raise RuntimeError(f"[{cat}] Could not find VALUE/GRIDCODE in snapped points.")

#     pp_final_id = _ensure_field(snapped_pts, wet_id_f, "LONG", fallbacks=("CWID","CW_ID","CW_Id"))
#     gp(f"[{cat}] Calculate CW_Id on snapped points", arcpy.management.CalculateField,
#        snapped_pts, pp_final_id, f"!{val_field}!", "PYTHON3")

#     uniq_pp = count_unique(snapped_pts, pp_final_id)
#     _log(f"[{cat}] unique snapped pourpoint IDs: {uniq_pp} (target {wet_n})")

#     # --- 7) Convert snapped points back to raster for Watershed
#     pp_snap_ras = os.path.join(pp_gdb, f"{cat}_pp_snap_ras")
#     _safe_delete(pp_snap_ras)
#     gp(f"[{cat}] PointToRaster snapped points", arcpy.conversion.PointToRaster,
#        snapped_pts, pp_final_id, pp_snap_ras, "MAXIMUM", "", cellsize)

#     # --- 8) Watershed raster (NO lake/stream raster masking here!)
#     ws_ras = os.path.join(ws_gdb, f"{cat}_ws_ras")
#     _safe_delete(ws_ras)
#     _log(f"▶ [{cat}] Watershed")
#     t0 = time.time()
#     sa.Watershed(D8_flow, pp_snap_ras).save(ws_ras)
#     _log(f"✅ DONE [{cat}] Watershed ({(time.time()-t0)/60:.2f} min)")

#     # --- 9) RasterToPolygon (NO raster masking)
#     ws_poly_raw = os.path.join(ws_gdb, f"{cat}_ws_poly_raw")
#     _safe_delete(ws_poly_raw)
#     gp(f"[{cat}] RasterToPolygon", arcpy.conversion.RasterToPolygon, ws_ras, ws_poly_raw, "NO_SIMPLIFY", "VALUE")

#     grid_field = _find_field(ws_poly_raw, ["GRIDCODE","GRID_CODE"])
#     if not grid_field:
#         raise RuntimeError(f"[{cat}] GRIDCODE missing in watershed polygons.")

#     ws_id_f = _ensure_field(ws_poly_raw, wet_id_f, "LONG", fallbacks=("CWID","CW_ID","CW_Id"))
#     gp(f"[{cat}] Calculate CW_Id on watershed polygons", arcpy.management.CalculateField,
#        ws_poly_raw, ws_id_f, f"!{grid_field}!", "PYTHON3")

#     ws_poly = os.path.join(ws_gdb, f"{cat}_ws_poly")
#     _safe_delete(ws_poly)
#     gp(f"[{cat}] Dissolve watersheds by CW_Id", arcpy.management.Dissolve, ws_poly_raw, ws_poly, ws_id_f)

#     ws_before = count_unique(ws_poly, ws_id_f)
#     _log(f"[{cat}] watersheds BEFORE clipping unique IDs: {ws_before} (target {wet_n})")

#     # --- 10) CLIP OUT ONLY overlap parts (lake + stream) as polygons
#     tmp_no_lake = os.path.join(ws_gdb, f"{cat}_ws_no_lake")
#     tmp_no_stream = os.path.join(ws_gdb, f"{cat}_ws_no_stream")
#     _safe_delete(tmp_no_lake); _safe_delete(tmp_no_stream)

#     gp(f"[{cat}] Erase lake from watersheds (clip)", arcpy.analysis.Erase, ws_poly, Lake_tgt, tmp_no_lake)
#     gp(f"[{cat}] Erase stream mask from watersheds (clip)", arcpy.analysis.Erase, tmp_no_lake, inStreamsWS_buf, tmp_no_stream)

#     _safe_delete(out_ws_lake)
#     gp(f"[{cat}] Dissolve final output (clip result)", arcpy.management.Dissolve, tmp_no_stream, out_ws_lake, ws_id_f)

#     ws_after = count_unique(out_ws_lake, ws_id_f)
#     _log(f"[{cat}] final watersheds AFTER clip unique IDs: {ws_after} (target {wet_n})")

#     # ============================================================
# # 11) REPAIR missing IDs (FAST VERSION)
# # ============================================================
# # 11) REPAIR missing IDs (ASSIGNMENT ONLY - FAST)
# #     This guarantees 1 feature per CW_Id without per-ID watershed rebuild.
# # ============================================================
# # ============================================================
# # 11) REPAIR missing IDs (ASSIGNMENT ONLY - FAST)
# #     This guarantees 1 feature per CW_Id without per-ID watershed rebuild.
# # ============================================================

#     final_ids = _idset(out_ws_lake, ws_id_f)
#     missing_ids = sorted(list(orig_ids - final_ids))
#     _log(f"[{cat}] Missing IDs after clip: {len(missing_ids)}")

#     if missing_ids:
#         # Extract pourpoints for missing IDs
#         pp_layer = f"lyr_{cat}_pp_inside"
#         arcpy.management.MakeFeatureLayer(pp_inside, pp_layer)

#         miss_pts = os.path.join(pp_gdb, f"{cat}_missing_pts")
#         _safe_delete(miss_pts)

#         arcpy.management.SelectLayerByAttribute(pp_layer, "CLEAR_SELECTION")
#         chunk = 900
#         for i in range(0, len(missing_ids), chunk):
#             sub = missing_ids[i:i+chunk]
#             where = f"{arcpy.AddFieldDelimiters(pp_layer, pp_id_f)} IN ({','.join(map(str, sub))})"
#             arcpy.management.SelectLayerByAttribute(pp_layer, "ADD_TO_SELECTION", where)

#         gp(f"[{cat}] Copy missing pourpoints", arcpy.management.CopyFeatures, pp_layer, miss_pts)
#         arcpy.management.Delete(pp_layer)

#         # Near -> closest existing final watershed polygon
#         gp(f"[{cat}] Near(missing pts -> final watersheds)", arcpy.analysis.Near, miss_pts, out_ws_lake)

#         # Build donor geometry lookup (OID -> geometry)
#         oid_field = arcpy.Describe(out_ws_lake).OIDFieldName
#         donor_geom = {}
#         with arcpy.da.SearchCursor(out_ws_lake, [oid_field, "SHAPE@"]) as cur:
#             for oid, geom in cur:
#                 donor_geom[int(oid)] = geom

#         # Create assigned FC (one polygon per missing CW_Id)
#         assigned_fc = os.path.join(ws_gdb, f"{cat}_assigned_missing")
#         _safe_delete(assigned_fc)
#         gp(f"[{cat}] Create assigned FC", arcpy.management.CreateFeatureclass,
#         ws_gdb, os.path.basename(assigned_fc), "POLYGON", None, "DISABLED", "DISABLED", D8_SR)

#         assigned_id = _ensure_field(assigned_fc, ws_id_f, "LONG", fallbacks=("CWID","CW_ID","CW_Id"))

#         near_fld = _find_field(miss_pts, ["NEAR_FID"])
#         n_assigned = 0
#         with arcpy.da.SearchCursor(miss_pts, [pp_id_f, near_fld]) as cur, \
#             arcpy.da.InsertCursor(assigned_fc, [assigned_id, "SHAPE@"]) as ic:
#             for cw, near_fid in cur:
#                 if near_fid is None or int(near_fid) < 0:
#                     continue
#                 geom = donor_geom.get(int(near_fid))
#                 if geom is None:
#                     continue
#                 ic.insertRow((int(cw), geom))
#                 n_assigned += 1

#         _log(f"[{cat}] Assigned polygons added: {n_assigned}")

#         if int(arcpy.management.GetCount(assigned_fc)[0]) > 0:
#             gp(f"[{cat}] Append assigned -> final", arcpy.management.Append, assigned_fc, out_ws_lake, "NO_TEST")

#             out_ws_lake_diss2 = os.path.join(ws_gdb, f"{cat}_final_diss2")
#             _safe_delete(out_ws_lake_diss2)

#             # Use PairwiseDissolve if available (faster)
#             if hasattr(arcpy.analysis, "PairwiseDissolve"):
#                 gp(f"[{cat}] PairwiseDissolve (enforce 1 per CW_Id)", arcpy.analysis.PairwiseDissolve,
#                 out_ws_lake, out_ws_lake_diss2, ws_id_f)
#             else:
#                 gp(f"[{cat}] Dissolve (enforce 1 per CW_Id)", arcpy.management.Dissolve,
#                 out_ws_lake, out_ws_lake_diss2, ws_id_f)

#             _safe_delete(out_ws_lake)
#             gp(f"[{cat}] Copy enforced final", arcpy.management.CopyFeatures, out_ws_lake_diss2, out_ws_lake)

#         _safe_delete(miss_pts)

#     ws_after_final = count_unique(out_ws_lake, ws_id_f)
#     _log(f"[{cat}] final watersheds AFTER assignment-repair: {ws_after_final} (target {len(orig_ids)})")


#     # --- 12) Add attributes safely (do not crash if locked)
#     calculate_area_m2(out_ws_lake, "WS_AREAM2")
#     add_xy_ll(out_ws_lake, prefix="WS")

#     _log(f"✅ final watershed: {cat} -> {os.path.basename(out_ws_lake)}")
#     _clear_locks()

# _log("\n🎉 DONE")


✅ flowacc: S:\Projects\Active\GLB_Nutrient_Transport\DEM_rasters\GLB_Bdry_buff10km_dem_fill_flowaccu.tif
✅ workspace: D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\CoastalWatersheds\Watershed_rasters\watersheds.gdb


<>:207: SyntaxWarning: invalid escape sequence '\)'
<>:207: SyntaxWarning: invalid escape sequence '\)'
C:\Users\abolmaal\AppData\Local\Temp\ipykernel_36968\1971050200.py:207: SyntaxWarning: invalid escape sequence '\)'
  All temp writes stay in ws_gdb (no C:\).


✅ scratch:   D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\CoastalWatersheds\Watershed_rasters\watersheds.gdb
✅ D8_flow CRS: NAD_1983_Great_Lakes_Basin_Albers (factoryCode=3174)

[inStreamsWS] input: D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Streamwatershed\PointWaterdhed_LH.shp
[inStreamsWS] sr: GCS_WGS_1984 | factoryCode=4326 | type=Geographic
[inStreamsWS] extent: XMin=929846.850 XMax=1166434.867 YMin=651452.122 YMax=1020243.911
▶ CopyFeatures inStreamsWS
✅ DONE CopyFeatures inStreamsWS (0.01 min)
[inStreamsWS] ⚠️ MISLABELED Geographic but coords are projected. SKIP Project; DefineProjection -> NAD_1983_Great_Lakes_Basin_Albers
▶ DefineProjection inStreamsWS
✅ DONE DefineProjection inStreamsWS (0.00 min)
▶ Copy to output inStreamsWS
✅ DONE Copy to output inStreamsWS (0.01 min)
▶ RepairGeometry stream mask
✅ DONE RepairGeometry stream mask (0.00 min)
▶ MultipartToSinglepart stream mask
✅ DONE MultipartToSinglepart stream mask (0.01 min)
▶ Dissolve stream mask
✅ DONE D

## What this cell does (end-to-end)

This cell builds **coastal watershed polygons for each coastal wetland ID (`CW_Id`)**, using a D8 flow–direction grid, while ensuring that any **parts overlapping Lake Huron and the Stream-Watershed mask are clipped out (removed as pieces, not whole features)**. All intermediate outputs are written into your **project geodatabases** (no `C:\temp` scratch).

---

### 1) Sets up safe workspaces (no C:\ temp)
- Creates/uses two file geodatabases:
  - `pourpoints.gdb` (stores pourpoints and intermediate rasters/points)
  - `watersheds.gdb` (stores intermediate and final watershed polygons)
- Forces `arcpy.env.workspace` and `arcpy.env.scratchWorkspace` to `watersheds.gdb`.
- Aligns processing to the D8 grid by setting:
  - `snapRaster = D8_flow`
  - `cellSize = D8_flow`
  - `extent = D8_flow`
  - `outputCoordinateSystem = D8_flow spatial reference`

**Result:** all rasters/vectors line up exactly with the D8 grid.

---

### 2) Builds the two “mask” polygons in the D8 coordinate system (once)
This happens once before looping over categories.

#### A) Stream-watershed mask (project + clean)
- Fixes CRS issues (handles “mislabeled geographic but projected coordinates” cases).
- Projects/defines the stream-watershed polygon into the D8 CRS.
- Repairs geometry, converts multipart to singlepart, then dissolves into one polygon.
- Buffers the dissolved stream mask by `stream_buf_m` meters (default: 60 m).

**Why buffer?** Helps remove tiny slivers and boundary artifacts when erasing.

#### B) Lake Huron polygon (project + clean)
- Fixes CRS issues (handles generic geographic degrees).
- Projects the lake polygon into the D8 CRS.
- Repairs geometry.
- Converts the lake polygon into a raster mask aligned to the D8 grid.
- Creates `flowacc_land = flowacc where NOT lake` using `SetNull`.

**Purpose of `flowacc_land`:** Pourpoints snap to *land* flow-accumulation cells, not lake cells.

---

### 3) For each inundation category (`high`, `low`, `surge`), it builds watersheds

#### Step 3.1 — Project wetlands into D8 CRS
- Projects the wetland feature class (for this category) into the D8 CRS.
- Ensures the `CW_Id` field exists.
- Repairs geometry.

#### Step 3.2 — Remove only the overlapped PART of wetlands inside stream-watershed mask
- Runs `Erase(wetlands, stream_mask_buffered)` and outputs `wet_no_stream`.

✅ This **does NOT delete entire wetlands**.  
It only removes the *pieces* of wetlands that fall inside the stream-watershed mask.

#### Step 3.3 — Create “land-only” wetlands (used to place pourpoints)
- Runs `Erase(wet_no_stream, LakeHuron)` to remove lake-covered pieces.
- Output: `wet_land`.

This is used only to create pourpoints on land.

#### Step 3.4 — Create pourpoints (potentially multiple per `CW_Id`)
- Runs `FeatureToPoint(wet_land, INSIDE)` to get a point inside each land polygon feature.
- Because this is done on **non-dissolved** polygons, a single `CW_Id` can generate **multiple pourpoints** if it has multiple polygons.

✅ All points preserve and carry `CW_Id`.

#### Step 3.5 — Fallback points for CW_Ids with no remaining land
Some `CW_Id` polygons might be completely removed by lake/stream erasing (e.g., “lake-only” or fully masked).

- The code detects CW_Ids that exist in the original wetlands but are missing after land-only erase.
- For those CW_Ids:
  - Dissolves the **original** wetlands by `CW_Id`
  - Creates **one fallback point per missing CW_Id** using `FeatureToPoint(INSIDE)`
  - Appends these fallback points into the main pourpoints feature class

✅ This prevents losing CW_Ids just because their wetland is fully in-lake or fully masked.

---

### 4) Snaps pourpoints to the land flow-accumulation grid
To generate hydrologically valid outlets:

1. Converts pourpoints to a raster (`PointToRaster`) with cell values = `CW_Id`
2. Snaps those raster points to high-flow-accumulation cells using:
   - `SnapPourPoint(pp_raster, flowacc_land, snap_dist_m)`
3. Converts snapped raster back to points (`RasterToPoint`)
4. Copies the snapped point raster VALUE/GRIDCODE into a proper `CW_Id` field

✅ After snapping, each snapped point has a valid `CW_Id`.

---

### 5) Builds watershed polygons from the snapped outlets
1. Converts snapped points back to raster (value = `CW_Id`)
2. Runs `sa.Watershed(D8_flow, snapped_outlet_raster)` to produce a watershed raster
3. Converts watershed raster to polygons (`RasterToPolygon`)
4. Copies raster zone value into `CW_Id`
5. Dissolves polygons by `CW_Id` so final watersheds are grouped by ID

**Important:** `Watershed()` assigns each D8 cell to exactly one outlet zone, so the resulting watershed zones generally do not overlap between CW_Ids.

---

### 6) Clips out only the overlapped PARTS of the watersheds (lake + stream)
After watersheds are created, the code removes overlap *as pieces*:

- `Erase(watersheds, LakeHuron)` → removes parts that fall inside the lake polygon
- `Erase(result, StreamMaskBuffered)` → removes parts that fall inside the buffered stream-watershed mask

✅ This **does NOT delete whole CW_Ids**.  
It removes only the overlapping portions, keeping the rest.

Then it dissolves again by `CW_Id` and saves to your final output path (`out_ws_lake`).

---

### 7) Fast “repair missing IDs” (assignment fallback)
Sometimes CW_Ids can still disappear (usually due to SnapPourPoint collisions or tiny/empty results after clipping).

If CW_Ids are missing after final clipping:
- The code selects the missing pourpoints
- Runs `Near(missing_pts, final_watersheds)` to find the nearest existing watershed polygon
- Copies that nearest polygon geometry and assigns it to the missing CW_Id
- Appends those assigned polygons to the final output
- Dissolves again by `CW_Id`

✅ This guarantees each missing CW_Id is represented, without re-running Watershed per ID.

(But note: this “assignment repair” copies geometry from a neighbor, so it is a pragmatic fallback—not a true hydrologic rebuild.)

---

### 8) Adds attributes to the final watershed polygons
For the final watershed output:
- Adds/updates `WS_AREAM2` (area in square meters)
- Adds centroid coordinates:
  - `WS_cx`, `WS_cy` (projected CRS)
  - `WS_lon`, `WS_lat` (WGS84 geographic)

---

## Summary of the main outputs you get
For each category (`high`, `low`, `surge`), you get:
- A final watershed polygon feature class where:
  - Each feature has `CW_Id`
  - Watershed pieces overlapping **Lake Huron** are removed
  - Watershed pieces overlapping **Stream-Watershed mask** are removed
  - Missing CW_Ids are optionally “repaired” via nearest-polygon assignment
  - Area + centroid fields are added

Note:
once we create pourpoints it rasterizes the pourpoints using CW_ids and if multiple pourpoint land in a same raster cell only one CW_id survide and watershed assigns each raster cell to one outlet values and the we dissovlve it by CW_Id.
So even if CW_Id had multiple pourpoints, the watershed output is effectively:

- sa.Watershed(D8_flow, pp_snap_ras)  # outlets are CW_Id values
- RasterToPolygon(...)
- Dissolve(..., dissolve_field=CW_Id)

“all cells that drain to any outlet that has value CW_Id”

then merged into one multipart polygon per CW_Id

✅ Final result: you do NOT get multiple coastal watersheds per CW_Id

In [10]:
# --- FINAL ROBUST CELL (STREAM ERASE ON WETLANDS + WATERSHEDS, NO LAKE ERASE) ---
# ✅ Wetlands: ERASE stream overlap portion (optionally buffered) -> keeps remaining wetland parts
# ✅ Watersheds: ERASE stream overlap portion (optionally buffered) -> keeps remaining watershed parts
# ✅ SnapPourPoint uses PP_ID (NOT CW_Id) to avoid losing CW_Ids
# ❌ No lake erase anywhere

import os, gc, time, sys
import arcpy
from arcpy import sa

arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False
arcpy.CheckOutExtension("Spatial")

# -------------------------
# REQUIRED: these must already exist in your notebook
# -------------------------
# outPourpoints, outWatersheds, inStreamsWatershed, D8_flow
# erase_buffer_avg/high/low/surge
# CoastalWatershed_* output paths

os.makedirs(outPourpoints, exist_ok=True)
os.makedirs(outWatersheds, exist_ok=True)

pp_gdb = os.path.join(outPourpoints, "pourpoints.gdb")
if not arcpy.Exists(pp_gdb):
    arcpy.management.CreateFileGDB(outPourpoints, "pourpoints.gdb")

ws_gdb = os.path.join(outWatersheds, "watersheds.gdb")
if not arcpy.Exists(ws_gdb):
    arcpy.management.CreateFileGDB(outWatersheds, "watersheds.gdb")

arcpy.env.workspace = ws_gdb
arcpy.env.scratchWorkspace = ws_gdb

flowacc = r"S:\Projects\Active\GLB_Nutrient_Transport\DEM_rasters\GLB_Bdry_buff10km_dem_fill_flowaccu.tif"
print(f"✅ flowacc: {flowacc}", flush=True)

cats = {
    "avg":   (erase_buffer_avg,   CoastalWatershed_avg_erase_lakedrain_LakeHuron),
    "high":  (erase_buffer_high,  CoastalWatershed_high_erase_lakedrain_LakeHuron),
    "low":   (erase_buffer_low,   CoastalWatershed_low_erase_lakedrain_LakeHuron),
    "surge": (erase_buffer_surge, CoastalWatershed_surge_erase_lakedrain_LakeHuron),
}
CW_ID_FIELD = "CW_Id"

# -------------------------
# Align env to D8 grid
# -------------------------
D8_SR = arcpy.Describe(D8_flow).spatialReference
arcpy.env.snapRaster = D8_flow
arcpy.env.cellSize = D8_flow
arcpy.env.extent = D8_flow
arcpy.env.outputCoordinateSystem = D8_SR
cellsize = float(arcpy.Describe(D8_flow).meanCellWidth)

print(f"✅ workspace: {ws_gdb}", flush=True)
print(f"✅ scratch:   {ws_gdb}", flush=True)
print(f"✅ D8_flow CRS: {D8_SR.name} (factoryCode={D8_SR.factoryCode})", flush=True)

# ============================================================
# Helpers
# ============================================================
def _log(msg):
    print(msg, flush=True)
    sys.stdout.flush()

def _clear_locks():
    try:
        arcpy.ClearWorkspaceCache_management()
    except Exception:
        pass
    gc.collect()

def _safe_delete(p):
    try:
        if arcpy.Exists(p):
            arcpy.management.Delete(p)
    except Exception:
        _clear_locks()
        if arcpy.Exists(p):
            arcpy.management.Delete(p)

def has_rows(fc):
    try:
        return int(arcpy.management.GetCount(fc)[0]) > 0
    except Exception:
        return False

def _field_map_lower(fc):
    return {f.name.lower(): f.name for f in arcpy.ListFields(fc)}

def _find_field(fc, candidates):
    fmap = _field_map_lower(fc)
    for c in candidates:
        if c and c.lower() in fmap:
            return fmap[c.lower()]
    return None

def get_field_name_ci(fc, target_name):
    if not target_name:
        return None
    t = target_name.lower()
    for f in arcpy.ListFields(fc):
        if f.name.lower() == t:
            return f.name
    return None

def _is_shp(path):
    return isinstance(path, str) and path.lower().endswith(".shp")

def _ensure_field(fc, desired_name, field_type="LONG", fallbacks=()):
    if not desired_name or not str(desired_name).strip():
        desired_name = "CW_Id"

    existing = get_field_name_ci(fc, desired_name)
    if existing:
        return existing

    candidates = [desired_name] + list(fallbacks)
    for nm in candidates:
        if not nm:
            continue

        safe = arcpy.ValidateFieldName(nm, os.path.dirname(fc) if isinstance(fc, str) else "")
        if _is_shp(fc) and len(safe) > 10:
            safe = safe[:10]

        existing = get_field_name_ci(fc, safe)
        if existing:
            return existing

        try:
            arcpy.management.AddField(fc, safe, field_type)
            return get_field_name_ci(fc, safe) or safe
        except Exception:
            _clear_locks()
            continue

    raise RuntimeError(f"Cannot add field '{desired_name}' to {fc}")

def _idset(fc, id_field):
    fld = get_field_name_ci(fc, id_field) or _find_field(fc, [id_field])
    s = set()
    if (not fld) or (not has_rows(fc)):
        return s
    with arcpy.da.SearchCursor(fc, [fld]) as cur:
        for (v,) in cur:
            if v is not None:
                s.add(int(v))
    return s

def gp(label, func, *args, **kwargs):
    _log(f"▶ {label}")
    t0 = time.time()
    out = func(*args, **kwargs)
    _log(f"✅ DONE {label} ({(time.time()-t0)/60:.2f} min)")
    return out

# ---------- CRS FIX + PROJECT (TEMP WRITES INTO ws_gdb) ----------
def fix_define_and_project_to_gdb(in_fc, out_fc, out_sr, name="layer"):
    def _looks_like_degrees(ext):
        return (abs(ext.XMin) <= 180 and abs(ext.XMax) <= 180 and abs(ext.YMin) <= 90 and abs(ext.YMax) <= 90)
    def _looks_like_projected(ext):
        return (max(abs(ext.XMin), abs(ext.XMax), abs(ext.YMin), abs(ext.YMax)) > 1000)

    _safe_delete(out_fc)

    d = arcpy.Describe(in_fc)
    sr = d.spatialReference
    ext = d.extent

    _log(f"\n[{name}] input: {in_fc}")
    _log(f"[{name}] sr: {sr.name if sr else None} | factoryCode={getattr(sr,'factoryCode',None)} | type={getattr(sr,'type',None)}")
    _log(f"[{name}] extent: XMin={ext.XMin:.3f} XMax={ext.XMax:.3f} YMin={ext.YMin:.3f} YMax={ext.YMax:.3f}")

    tmp_copy = os.path.join(ws_gdb, f"tmp_{name}_copy")
    _safe_delete(tmp_copy)

    with arcpy.EnvManager(outputCoordinateSystem=None, extent=None, snapRaster=None, cellSize=None):
        gp(f"CopyFeatures {name}", arcpy.management.CopyFeatures, in_fc, tmp_copy)

    d2 = arcpy.Describe(tmp_copy)
    sr2 = d2.spatialReference
    ext2 = d2.extent

    mislabeled_projected = (
        sr2 is not None
        and getattr(sr2, "type", None) == "Geographic"
        and _looks_like_projected(ext2)
        and not _looks_like_degrees(ext2)
    )

    if mislabeled_projected:
        _log(f"[{name}] ⚠️ MISLABELED Geographic but coords are projected. DefineProjection -> {out_sr.name}")
        gp(f"DefineProjection {name}", arcpy.management.DefineProjection, tmp_copy, out_sr)
        gp(f"Copy to output {name}", arcpy.management.CopyFeatures, tmp_copy, out_fc)
        _safe_delete(tmp_copy)
        return out_fc

    # Normal Project
    tx = None
    try:
        txs = arcpy.ListTransformations(sr2, out_sr)
        tx = txs[0] if txs else None
    except Exception:
        tx = None

    if tx:
        gp(f"Project {name}", arcpy.management.Project, tmp_copy, out_fc, out_sr, tx)
    else:
        gp(f"Project {name}", arcpy.management.Project, tmp_copy, out_fc, out_sr)

    _safe_delete(tmp_copy)
    return out_fc

# ============================================================
# 0) Build projected STREAM mask ONCE (stored in ws_gdb)
# ============================================================
inStreamsWS_tgt = os.path.join(ws_gdb, "inStreamsWS_tgt")
fix_define_and_project_to_gdb(inStreamsWatershed, inStreamsWS_tgt, D8_SR, name="inStreamsWS")
gp("RepairGeometry stream mask", arcpy.management.RepairGeometry, inStreamsWS_tgt)

inStreams_single = os.path.join(ws_gdb, "inStreamsWS_single")
_safe_delete(inStreams_single)
gp("MultipartToSinglepart stream mask", arcpy.management.MultipartToSinglepart, inStreamsWS_tgt, inStreams_single)

inStreamsWS_tgt_diss = os.path.join(ws_gdb, "inStreamsWS_tgt_diss")
_safe_delete(inStreamsWS_tgt_diss)
gp("Dissolve stream mask", arcpy.management.Dissolve, inStreams_single, inStreamsWS_tgt_diss)

# ---- Choose how aggressive the wetland/watershed removal should be ----
# If you want EXACT removal only where it truly overlaps the stream watershed, set stream_buf_m = 0.
# If you want "remove within distance", set >0 (e.g., 60m, 120m, etc.).
stream_buf_m = 60

if stream_buf_m > 0:
    inStreamsWS_mask = os.path.join(ws_gdb, f"inStreamsWS_buf{stream_buf_m}m")
    _safe_delete(inStreamsWS_mask)
    gp("Buffer stream mask", arcpy.analysis.Buffer, inStreamsWS_tgt_diss, inStreamsWS_mask,
       f"{stream_buf_m} Meters", "FULL", "ROUND", "ALL")
else:
    inStreamsWS_mask = inStreamsWS_tgt_diss
    _log("▶ stream_buf_m=0 -> using stream watershed polygons WITHOUT buffer")

# ============================================================
# MAIN LOOP
# ============================================================
for cat, (wet_fc, out_ws_final) in cats.items():

    _log(f"\n==================== {cat.upper()} ====================")

    # --- 1) Project wetlands to D8 SR
    wet_tgt = os.path.join(ws_gdb, f"{cat}_wet_tgt")
    _safe_delete(wet_tgt)
    gp(f"[{cat}] Project wetlands", arcpy.management.Project, wet_fc, wet_tgt, D8_SR)

    wet_id_f = _ensure_field(wet_tgt, CW_ID_FIELD, "LONG", fallbacks=("CWID","CW_ID","CW_Id"))
    gp(f"[{cat}] RepairGeometry wetlands", arcpy.management.RepairGeometry, wet_tgt)

    orig_ids = _idset(wet_tgt, wet_id_f)
    _log(f"[{cat}] unique CW_Ids in ORIGINAL wetlands: {len(orig_ids)}")

    # --- 2) ERASE wetlands by stream watershed (THIS is what you said you need)
    wet_no_stream = os.path.join(ws_gdb, f"{cat}_wet_no_stream")
    _safe_delete(wet_no_stream)
    if hasattr(arcpy.analysis, "PairwiseErase"):
        gp(f"[{cat}] PairwiseErase wetlands ∖ streammask (partial)", arcpy.analysis.PairwiseErase, wet_tgt, inStreamsWS_mask, wet_no_stream)
    else:
        gp(f"[{cat}] Erase wetlands ∖ streammask (partial)", arcpy.analysis.Erase, wet_tgt, inStreamsWS_mask, wet_no_stream)

    ns_ids = _idset(wet_no_stream, wet_id_f)
    _log(f"[{cat}] CW_Ids remaining after wetland stream-erase: {len(ns_ids)}")

    # --- 3) Pourpoints from wet_no_stream (stream-clean wetlands)
    pp_inside = os.path.join(pp_gdb, f"{cat}_pp_inside")
    _safe_delete(pp_inside)
    gp(f"[{cat}] FeatureToPoint INSIDE (stream-clean wetlands)", arcpy.management.FeatureToPoint, wet_no_stream, pp_inside, "INSIDE")

    pp_cw_f = _ensure_field(pp_inside, wet_id_f, "LONG", fallbacks=("CWID","CW_ID","CW_Id"))

    # --- 4) Fallback pourpoints for CW_Ids that lost ALL wetland area after erase
    missing_ids_for_points = sorted(list(orig_ids - ns_ids))
    if missing_ids_for_points:
        _log(f"⚠️ [{cat}] {len(missing_ids_for_points)} CW_Ids have NO wetland left after stream-erase; adding fallback points from ORIGINAL wetlands.")

        wet_orig_diss = os.path.join(ws_gdb, f"{cat}_wet_orig_diss")
        _safe_delete(wet_orig_diss)
        gp(f"[{cat}] Dissolve ORIGINAL wetlands by CW_Id (fallback)", arcpy.management.Dissolve, wet_tgt, wet_orig_diss, wet_id_f)

        wet_missing_poly = os.path.join(ws_gdb, f"{cat}_wet_missing_poly")
        _safe_delete(wet_missing_poly)

        lyr = f"lyr_{cat}_missing"
        arcpy.management.MakeFeatureLayer(wet_orig_diss, lyr)
        arcpy.management.SelectLayerByAttribute(lyr, "CLEAR_SELECTION")

        chunk = 900
        for i in range(0, len(missing_ids_for_points), chunk):
            sub = missing_ids_for_points[i:i+chunk]
            where = f"{arcpy.AddFieldDelimiters(lyr, wet_id_f)} IN ({','.join(map(str, sub))})"
            arcpy.management.SelectLayerByAttribute(lyr, "ADD_TO_SELECTION", where)

        gp(f"[{cat}] Copy fallback polys", arcpy.management.CopyFeatures, lyr, wet_missing_poly)
        arcpy.management.Delete(lyr)

        pp_fallback = os.path.join(pp_gdb, f"{cat}_pp_fallback")
        _safe_delete(pp_fallback)
        gp(f"[{cat}] FeatureToPoint INSIDE (fallback)", arcpy.management.FeatureToPoint, wet_missing_poly, pp_fallback, "INSIDE")

        gp(f"[{cat}] Append fallback points", arcpy.management.Append, pp_fallback, pp_inside, "NO_TEST")

        _safe_delete(pp_fallback)
        _safe_delete(wet_missing_poly)
        _safe_delete(wet_orig_diss)

    # Optional de-dup
    try:
        gp(f"[{cat}] DeleteIdentical points (Shape)", arcpy.management.DeleteIdentical, pp_inside, ["Shape"])
    except Exception:
        pass

    # --- 5) Assign unique PP_ID per point (prevents CW_Id loss)
    PP_ID = _ensure_field(pp_inside, "PP_ID", "LONG", fallbacks=("PPID",))
    oid = arcpy.Describe(pp_inside).OIDFieldName
    with arcpy.da.UpdateCursor(pp_inside, [oid, PP_ID]) as cur:
        i = 1
        for row in cur:
            row[1] = i
            cur.updateRow(row)
            i += 1

    # --- 6) SnapPourPoint on PP_ID raster (NOT CW_Id raster)
    pp_ras = os.path.join(pp_gdb, f"{cat}_pp_ras")
    _safe_delete(pp_ras)
    gp(f"[{cat}] PointToRaster pourpoints (PP_ID)", arcpy.conversion.PointToRaster, pp_inside, PP_ID, pp_ras, "MAXIMUM", "", cellsize)

    snapped_pp_ras = os.path.join(pp_gdb, f"{cat}_pp_snapped_ras")
    _safe_delete(snapped_pp_ras)
    snap_dist_m = 150
    _log(f"[{cat}] SnapPourPoint distance = {snap_dist_m} m (PP_ID raster)")
    sa.SnapPourPoint(sa.Raster(pp_ras), sa.Raster(flowacc), snap_dist_m).save(snapped_pp_ras)

    # --- 7) Watershed using snapped PP_ID raster
    ws_ras = os.path.join(ws_gdb, f"{cat}_ws_ras")
    _safe_delete(ws_ras)
    _log(f"▶ [{cat}] Watershed")
    t0 = time.time()
    sa.Watershed(D8_flow, snapped_pp_ras).save(ws_ras)
    _log(f"✅ DONE [{cat}] Watershed ({(time.time()-t0)/60:.2f} min)")

    # --- 8) RasterToPolygon (GRIDCODE = PP_ID), join PP_ID -> CW_Id, then dissolve by CW_Id
    ws_poly_raw = os.path.join(ws_gdb, f"{cat}_ws_poly_raw")
    _safe_delete(ws_poly_raw)
    gp(f"[{cat}] RasterToPolygon", arcpy.conversion.RasterToPolygon, ws_ras, ws_poly_raw, "NO_SIMPLIFY", "VALUE")

    grid_field = _find_field(ws_poly_raw, ["GRIDCODE","GRID_CODE"])
    if not grid_field:
        raise RuntimeError(f"[{cat}] GRIDCODE missing in watershed polygons.")

    PP_SRC = _ensure_field(ws_poly_raw, "PP_ID", "LONG", fallbacks=("PPID",))
    gp(f"[{cat}] Calculate PP_ID on ws polys", arcpy.management.CalculateField, ws_poly_raw, PP_SRC, f"!{grid_field}!", "PYTHON3")

    gp(f"[{cat}] JoinField ws_poly_raw.PP_ID -> pp_inside.PP_ID to bring CW_Id",
       arcpy.management.JoinField, ws_poly_raw, PP_SRC, pp_inside, PP_ID, [pp_cw_f])

    ws_cw = get_field_name_ci(ws_poly_raw, pp_cw_f) or get_field_name_ci(ws_poly_raw, CW_ID_FIELD)
    if not ws_cw:
        raise RuntimeError(f"[{cat}] CW_Id not found after JoinField. Fields: {[f.name for f in arcpy.ListFields(ws_poly_raw)]}")

    ws_poly = os.path.join(ws_gdb, f"{cat}_ws_poly")
    _safe_delete(ws_poly)
    gp(f"[{cat}] Dissolve watersheds by CW_Id", arcpy.management.Dissolve, ws_poly_raw, ws_poly, ws_cw)

    _log(f"[{cat}] watersheds BEFORE stream erase unique CW_Ids: {len(_idset(ws_poly, ws_cw))} (orig wetlands {len(orig_ids)})")

    # --- 9) ERASE watersheds by stream watershed (partial)
    tmp_no_stream = os.path.join(ws_gdb, f"{cat}_ws_no_stream")
    _safe_delete(tmp_no_stream)

    if hasattr(arcpy.analysis, "PairwiseErase"):
        gp(f"[{cat}] PairwiseErase watersheds ∖ streammask (partial)", arcpy.analysis.PairwiseErase, ws_poly, inStreamsWS_mask, tmp_no_stream)
    else:
        gp(f"[{cat}] Erase watersheds ∖ streammask (partial)", arcpy.analysis.Erase, ws_poly, inStreamsWS_mask, tmp_no_stream)

    _safe_delete(out_ws_final)
    if has_rows(tmp_no_stream):
        gp(f"[{cat}] Dissolve final output", arcpy.management.Dissolve, tmp_no_stream, out_ws_final, ws_cw)
    else:
        gp(f"[{cat}] Copy empty output", arcpy.management.CopyFeatures, tmp_no_stream, out_ws_final)

    _log(f"[{cat}] final watersheds AFTER stream erase unique CW_Ids: {len(_idset(out_ws_final, ws_cw))} (orig wetlands {len(orig_ids)})")

    _clear_locks()

_log("\n🎉 DONE")


✅ flowacc: S:\Projects\Active\GLB_Nutrient_Transport\DEM_rasters\GLB_Bdry_buff10km_dem_fill_flowaccu.tif
✅ workspace: D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\CoastalWatersheds\Watershed_rasters\watersheds.gdb
✅ scratch:   D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\CoastalWatersheds\Watershed_rasters\watersheds.gdb
✅ D8_flow CRS: NAD_1983_Great_Lakes_Basin_Albers (factoryCode=3174)

[inStreamsWS] input: D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Streamwatershed\PointWaterdhed_LH.shp
[inStreamsWS] sr: GCS_WGS_1984 | factoryCode=4326 | type=Geographic
[inStreamsWS] extent: XMin=929846.850 XMax=1166434.867 YMin=651452.122 YMax=1020243.911
▶ CopyFeatures inStreamsWS
✅ DONE CopyFeatures inStreamsWS (0.01 min)
[inStreamsWS] ⚠️ MISLABELED Geographic but coords are projected. DefineProjection -> NAD_1983_Great_Lakes_Basin_Albers
▶ DefineProjection inStreamsWS
✅ DONE DefineProjection inStreamsWS (0.00 min)
▶ Copy to output inStreamsWS
✅ DONE Copy to output inStreams

In [7]:
# --- FINAL ROBUST CELL (OPTION A, UPDATED v7)
# Option A = boundary-based pourpoints (NOT interior FeatureToPoint), Top-N per CW_Id.
#
# What this version fixes (based on your AVG log):
#  1) ✅ Pourpoints coverage: uses wet_land boundary + fallback boundary points from ORIGINAL wetlands
#     so pourpoints CW_Ids present == original CW_Ids.
#  2) ✅ SnapPourPoint: two-pass snapping (missing-only pass with larger distance).
#  3) ✅ BIG missing watersheds BEFORE clip: fixes SnapPourPoint collisions (multiple CW_Ids snapped to same cell),
#     which otherwise get dropped at PointToRaster and never become watersheds.
#  4) ✅ BIG missing watersheds AFTER clip: optional "clip fallback" to keep CW_Id if clip erases entire polygon.
#
# IMPORTANT CHOICE:
#  - If you truly want "remove overlap parts no matter what", then you must accept that some CW_Ids disappear
#    after clip if the whole polygon is inside the clip mask. In that case, set CLIP_FALLBACK=False below.
#  - If you want to keep CW_Id coverage, set CLIP_FALLBACK=True (default).

import os, gc, time, sys, math
import arcpy
from arcpy import sa

arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False
arcpy.CheckOutExtension("Spatial")

# -------------------------
# REQUIRED (must exist in notebook)
# -------------------------
# outPourpoints, outWatersheds, inStreamsWatershed, Lake_Huron, D8_flow
# erase_buffer_avg/high/low/surge
# CoastalWatershed_avg_erase_lakedrain, CoastalWatershed_avg_erase_lakedrain_LakeHuron
# CoastalWatershed_high_erase_lakedrain, CoastalWatershed_high_erase_lakedrain_LakeHuron
# CoastalWatershed_low_erase_lakedrain,  CoastalWatershed_low_erase_lakedrain_LakeHuron
# CoastalWatershed_surge_erase_lakedrain, CoastalWatershed_surge_erase_lakedrain_LakeHuron

os.makedirs(outPourpoints, exist_ok=True)
os.makedirs(outWatersheds, exist_ok=True)

pp_gdb = os.path.join(outPourpoints, "pourpoints.gdb")
if not arcpy.Exists(pp_gdb):
    arcpy.management.CreateFileGDB(outPourpoints, "pourpoints.gdb")

ws_gdb = os.path.join(outWatersheds, "watersheds.gdb")
if not arcpy.Exists(ws_gdb):
    arcpy.management.CreateFileGDB(outWatersheds, "watersheds.gdb")

# Force all workspace/scratch into ws_gdb (avoid C:\ temp)
arcpy.env.workspace = ws_gdb
arcpy.env.scratchWorkspace = ws_gdb

# -------------------------
# Inputs
# -------------------------
flowacc = r"S:\Projects\Active\GLB_Nutrient_Transport\DEM_rasters\GLB_Bdry_buff10km_dem_fill_flowaccu.tif"
print(f"✅ flowacc: {flowacc}", flush=True)

cats = {
    "avg":   (erase_buffer_avg,   CoastalWatershed_avg_erase_lakedrain,   CoastalWatershed_avg_erase_lakedrain_LakeHuron),
    "high":  (erase_buffer_high,  CoastalWatershed_high_erase_lakedrain,  CoastalWatershed_high_erase_lakedrain_LakeHuron),
    "low":   (erase_buffer_low,   CoastalWatershed_low_erase_lakedrain,   CoastalWatershed_low_erase_lakedrain_LakeHuron),
    "surge": (erase_buffer_surge, CoastalWatershed_surge_erase_lakedrain, CoastalWatershed_surge_erase_lakedrain_LakeHuron),
}

CW_ID_FIELD = "CW_Id"

# -------------------------
# Align env to D8 grid
# -------------------------
D8_SR = arcpy.Describe(D8_flow).spatialReference
arcpy.env.snapRaster = D8_flow
arcpy.env.cellSize   = D8_flow
arcpy.env.extent     = D8_flow
arcpy.env.outputCoordinateSystem = D8_SR
cellsize = float(arcpy.Describe(D8_flow).meanCellWidth)

print(f"✅ workspace: {ws_gdb}", flush=True)
print(f"✅ scratch:   {ws_gdb}", flush=True)
print(f"✅ D8_flow CRS: {D8_SR.name} (factoryCode={D8_SR.factoryCode})", flush=True)

# -------------------------
# Tunables
# -------------------------
TOP_N_PER_CWID = 1
snap_dist_1 = 150
snap_dist_2 = 2000          # missing-only pass; try 3000 if still missing snapped IDs
stream_buf_m = 60
CLIP_FALLBACK = True        # keep CW_Id coverage if clip erases everything
COLLISION_REPAIR = True     # repair snapped-cell collisions BEFORE watershed
COLLISION_RESNAP_DIST = 300 # how far to re-snap duplicates (>=snap_dist_1)

# ============================================================
# Helpers
# ============================================================
def _log(msg):
    print(msg, flush=True)
    sys.stdout.flush()

def _clear_locks():
    try:
        arcpy.ClearWorkspaceCache_management()
    except Exception:
        pass
    gc.collect()

def _safe_delete(p):
    try:
        if arcpy.Exists(p):
            arcpy.management.Delete(p)
    except Exception:
        _clear_locks()
        if arcpy.Exists(p):
            arcpy.management.Delete(p)

def _field_map_lower(fc):
    return {f.name.lower(): f.name for f in arcpy.ListFields(fc)}

def _find_field(fc, candidates):
    fmap = _field_map_lower(fc)
    for c in candidates:
        if c and c.lower() in fmap:
            return fmap[c.lower()]
    return None

def get_field_name_ci(fc, target_name):
    if not target_name:
        return None
    t = target_name.lower()
    for f in arcpy.ListFields(fc):
        if f.name.lower() == t:
            return f.name
    return None

def _ensure_field(fc, desired_name, field_type="LONG", fallbacks=()):
    if not desired_name or not str(desired_name).strip():
        desired_name = "CW_Id"
    existing = get_field_name_ci(fc, desired_name)
    if existing:
        return existing
    for nm in [desired_name] + list(fallbacks):
        if not nm:
            continue
        safe = arcpy.ValidateFieldName(nm, os.path.dirname(fc) if isinstance(fc, str) else "")
        if isinstance(fc, str) and fc.lower().endswith(".shp") and len(safe) > 10:
            safe = safe[:10]
        existing = get_field_name_ci(fc, safe)
        if existing:
            return existing
        try:
            arcpy.management.AddField(fc, safe, field_type)
            return get_field_name_ci(fc, safe) or safe
        except Exception:
            _clear_locks()
            continue
    raise RuntimeError(f"Cannot add field '{desired_name}' to {fc}")

def _idset(fc, id_field):
    fld = get_field_name_ci(fc, id_field) or _find_field(fc, [id_field])
    s = set()
    with arcpy.da.SearchCursor(fc, [fld]) as cur:
        for (v,) in cur:
            if v is not None:
                s.add(int(v))
    return s

def count_unique(fc, id_field):
    return len(_idset(fc, id_field))

def calculate_area_m2(fc, field="WS_AREAM2"):
    try:
        field = _ensure_field(fc, field, "DOUBLE", fallbacks=("AREA_M2","A_M2","AREA"))
        arcpy.management.CalculateGeometryAttributes(fc, [[field, "AREA"]], area_unit="SQUARE_METERS")
    except Exception as e:
        _log(f"⚠️ area field skipped: {e}")
    return field

def add_xy_ll(fc, prefix="WS"):
    try:
        cx = _ensure_field(fc, f"{prefix}_cx", "DOUBLE", fallbacks=(f"{prefix}X",))
        cy = _ensure_field(fc, f"{prefix}_cy", "DOUBLE", fallbacks=(f"{prefix}Y",))
        arcpy.management.CalculateField(fc, cx, "!SHAPE.centroid.X!", "PYTHON3")
        arcpy.management.CalculateField(fc, cy, "!SHAPE.centroid.Y!", "PYTHON3")

        lon = _ensure_field(fc, f"{prefix}_lon", "DOUBLE", fallbacks=(f"{prefix}LON",))
        lat = _ensure_field(fc, f"{prefix}_lat", "DOUBLE", fallbacks=(f"{prefix}LAT",))
        arcpy.management.CalculateGeometryAttributes(
            fc,
            [[lat, "CENTROID_Y"], [lon, "CENTROID_X"]],
            coordinate_system=arcpy.SpatialReference(4326),
            coordinate_format="DD"
        )
    except Exception as e:
        _log(f"⚠️ xy/ll fields skipped: {e}")

def gp(label, func, *args, **kwargs):
    _log(f"▶ {label}")
    t0 = time.time()
    out = func(*args, **kwargs)
    _log(f"✅ DONE {label} ({(time.time()-t0)/60:.2f} min)")
    return out

# ============================================================
# CRS Fixer (mislabeled geographic -> define/project safely)
# ============================================================
def fix_define_and_project_to_gdb(in_fc, out_fc, out_sr, assumed_src_if_mislabeled=None, name="layer"):
    def _looks_like_degrees(ext):
        return (abs(ext.XMin) <= 180 and abs(ext.XMax) <= 180 and abs(ext.YMin) <= 90 and abs(ext.YMax) <= 90)
    def _looks_like_projected(ext):
        return (max(abs(ext.XMin), abs(ext.XMax), abs(ext.YMin), abs(ext.YMax)) > 1000)
    def _pick_transform(in_sr, out_sr):
        try:
            tx = arcpy.ListTransformations(in_sr, out_sr)
            return tx[0] if tx else None
        except Exception:
            return None

    _safe_delete(out_fc)

    d = arcpy.Describe(in_fc)
    sr = d.spatialReference
    ext = d.extent

    _log(f"\n[{name}] input: {in_fc}")
    _log(f"[{name}] sr: {sr.name if sr else None} | factoryCode={getattr(sr,'factoryCode',None)} | type={getattr(sr,'type',None)}")
    _log(f"[{name}] extent: XMin={ext.XMin:.3f} XMax={ext.XMax:.3f} YMin={ext.YMin:.3f} YMax={ext.YMax:.3f}")

    tmp_copy = os.path.join(ws_gdb, f"tmp_{name}_copy")
    _safe_delete(tmp_copy)

    with arcpy.EnvManager(outputCoordinateSystem=None, extent=None, snapRaster=None, cellSize=None):
        gp(f"CopyFeatures {name}", arcpy.management.CopyFeatures, in_fc, tmp_copy)

    d2 = arcpy.Describe(tmp_copy)
    sr2 = d2.spatialReference
    ext2 = d2.extent

    mislabeled_projected = (
        sr2 is not None and getattr(sr2, "type", None) == "Geographic"
        and _looks_like_projected(ext2) and not _looks_like_degrees(ext2)
    )
    if mislabeled_projected:
        _log(f"[{name}] ⚠️ MISLABELED Geographic but coords are projected. DefineProjection -> {out_sr.name}")
        gp(f"DefineProjection {name}", arcpy.management.DefineProjection, tmp_copy, out_sr)
        gp(f"Copy to output {name}", arcpy.management.CopyFeatures, tmp_copy, out_fc)
        _safe_delete(tmp_copy)
        return out_fc

    generic_degrees = (
        sr2 is not None and getattr(sr2, "type", None) == "Geographic"
        and _looks_like_degrees(ext2)
        and getattr(sr2, "factoryCode", 0) in (0, None)
    )
    if generic_degrees:
        if assumed_src_if_mislabeled is None:
            assumed_src_if_mislabeled = arcpy.SpatialReference(4326)
        _log(f"[{name}] ⚠️ GENERIC geographic degrees. DefineProjection -> EPSG:4326 then Project.")
        gp(f"DefineProjection {name}", arcpy.management.DefineProjection, tmp_copy, assumed_src_if_mislabeled)

    sr_fixed = arcpy.Describe(tmp_copy).spatialReference
    if sr_fixed and (sr_fixed.factoryCode == out_sr.factoryCode) and (sr_fixed.name == out_sr.name):
        _log(f"[{name}] Already in target CRS. Copy only.")
        gp(f"Copy to output {name}", arcpy.management.CopyFeatures, tmp_copy, out_fc)
        _safe_delete(tmp_copy)
        return out_fc

    transform = _pick_transform(sr_fixed, out_sr)
    _log(f"[{name}] Project -> {out_sr.name} | transform={transform}")
    if transform:
        gp(f"Project {name}", arcpy.management.Project, tmp_copy, out_fc, out_sr, transform)
    else:
        gp(f"Project {name}", arcpy.management.Project, tmp_copy, out_fc, out_sr)

    _safe_delete(tmp_copy)
    return out_fc

def polygon_to_mask_raster(poly_fc, out_ras, value=1):
    _safe_delete(out_ras)
    fld = "MASKVAL"
    if fld not in [f.name for f in arcpy.ListFields(poly_fc)]:
        arcpy.management.AddField(poly_fc, fld, "SHORT")
        arcpy.management.CalculateField(poly_fc, fld, value, "PYTHON3")
    gp(f"PolygonToRaster {os.path.basename(out_ras)}", arcpy.conversion.PolygonToRaster,
       poly_fc, fld, out_ras, "CELL_CENTER", "", cellsize)
    return out_ras

# ============================================================
# Option A pourpoints: boundary vertices TOP-N by flowacc (or boundary fallback)
# (Fixed: never Sort by OID field type; uses SQL ORDER BY or OID_LONG)
# ============================================================
def pourpoints_boundary_topN_flowacc_or_boundary(wet_poly_fc, out_points_fc, id_field, flowacc_raster, scratch_gdb, top_n=5):
    tmp_line   = os.path.join(scratch_gdb, "tmp_wet_boundary_line")
    tmp_vtx    = os.path.join(scratch_gdb, "tmp_wet_boundary_vtx")
    tmp_join   = os.path.join(scratch_gdb, "tmp_vtx_join_cwid")
    tmp_vals   = os.path.join(scratch_gdb, "tmp_vtx_flowacc")
    tmp_keep   = os.path.join(scratch_gdb, "tmp_vtx_keep")
    tmp_sorted = os.path.join(scratch_gdb, "tmp_vtx_sorted")

    for p in [tmp_line, tmp_vtx, tmp_join, tmp_vals, tmp_keep, tmp_sorted, out_points_fc]:
        _safe_delete(p)

    gp("PolygonToLine (wet boundary)", arcpy.management.PolygonToLine, wet_poly_fc, tmp_line)
    gp("FeatureVerticesToPoints (ALL)", arcpy.management.FeatureVerticesToPoints, tmp_line, tmp_vtx, "ALL")

    gp("SpatialJoin vertices -> wetlands (attach CW_Id)", arcpy.analysis.SpatialJoin,
       tmp_vtx, wet_poly_fc, tmp_join,
       "JOIN_ONE_TO_ONE", "KEEP_COMMON", None, "INTERSECT")

    id_on_pts = get_field_name_ci(tmp_join, id_field) or _find_field(tmp_join, [id_field, "CWID", "CW_ID", "CW_Id"])
    if not id_on_pts:
        raise RuntimeError(f"After SpatialJoin, could not find '{id_field}' on boundary points.")

    gp("ExtractValuesToPoints (flowacc)", sa.ExtractValuesToPoints,
       tmp_join, flowacc_raster, tmp_vals, "NONE", "VALUE_ONLY")

    flds = [f.name for f in arcpy.ListFields(tmp_vals)]
    val_field = None
    for cand in ["RASTERVALU", "RASTERVALU1", "VALUE", "GridCode", "GRIDCODE"]:
        if cand in flds:
            val_field = cand
            break
    if val_field is None:
        float_fields = [f.name for f in arcpy.ListFields(tmp_vals) if f.type in ("Double","Single")]
        val_field = float_fields[0] if float_fields else None

    if val_field:
        lyr = "lyr_flowacc"
        arcpy.management.MakeFeatureLayer(tmp_vals, lyr)
        where_valid = f"{arcpy.AddFieldDelimiters(lyr, val_field)} IS NOT NULL"
        gp("Select non-null flowacc", arcpy.management.SelectLayerByAttribute, lyr, "NEW_SELECTION", where_valid)
        gp("Copy non-null flowacc pts", arcpy.management.CopyFeatures, lyr, tmp_keep)
        arcpy.management.Delete(lyr)

    # Fallback: all flowacc NULL -> boundary vertices (no ranking)
    if (not val_field) or int(arcpy.management.GetCount(tmp_keep)[0]) == 0:
        _log("⚠️ boundary flowacc sampling returned ALL NULL; using boundary vertices (no flowacc ranking) as pourpoints.")
        oid = arcpy.Describe(tmp_join).OIDFieldName

        gp("Create output pourpoints FC", arcpy.management.CreateFeatureclass,
           os.path.dirname(out_points_fc), os.path.basename(out_points_fc),
           "POINT", tmp_join, "DISABLED", "DISABLED", D8_SR)

        out_id = _ensure_field(out_points_fc, id_field, "LONG", fallbacks=("CWID","CW_ID","CW_Id"))
        _ensure_field(out_points_fc, "FA", "DOUBLE", fallbacks=("FLOWACC","FLOW_ACC"))

        used_sql = False
        try:
            sql = (None, f"ORDER BY {id_on_pts}, {oid}")
            with arcpy.da.SearchCursor(tmp_join, [id_on_pts, "SHAPE@"], sql_clause=sql) as cur, \
                 arcpy.da.InsertCursor(out_points_fc, [out_id, "SHAPE@"]) as ic:
                counts = {}
                for cw, geom in cur:
                    if cw is None:
                        continue
                    cw = int(cw)
                    counts.setdefault(cw, 0)
                    if counts[cw] < int(top_n):
                        ic.insertRow((cw, geom))
                        counts[cw] += 1
            used_sql = True
        except Exception as e:
            _log(f"⚠️ SQL ORDER BY failed ({e}). Falling back to Sort using a LONG copy of OID.")

        if not used_sql:
            oid_long = _ensure_field(tmp_join, "OID_LONG", "LONG", fallbacks=("OIDL",))
            gp("Calc OID_LONG", arcpy.management.CalculateField, tmp_join, oid_long, f"!{oid}!", "PYTHON3")
            _safe_delete(tmp_sorted)
            gp("Sort boundary vertices (CW_Id asc, OID_LONG asc)", arcpy.management.Sort,
               tmp_join, tmp_sorted, [[id_on_pts, "ASCENDING"], [oid_long, "ASCENDING"]])

            with arcpy.da.SearchCursor(tmp_sorted, [id_on_pts, "SHAPE@"]) as cur, \
                 arcpy.da.InsertCursor(out_points_fc, [out_id, "SHAPE@"]) as ic:
                counts = {}
                for cw, geom in cur:
                    if cw is None:
                        continue
                    cw = int(cw)
                    counts.setdefault(cw, 0)
                    if counts[cw] < int(top_n):
                        ic.insertRow((cw, geom))
                        counts[cw] += 1

    else:
        gp("Sort (CW_Id asc, flowacc desc)", arcpy.management.Sort, tmp_keep, tmp_sorted,
           [[id_on_pts, "ASCENDING"], [val_field, "DESCENDING"]])

        gp("Create output pourpoints FC", arcpy.management.CreateFeatureclass,
           os.path.dirname(out_points_fc), os.path.basename(out_points_fc),
           "POINT", tmp_sorted, "DISABLED", "DISABLED", D8_SR)

        out_id = _ensure_field(out_points_fc, id_field, "LONG", fallbacks=("CWID","CW_ID","CW_Id"))
        out_fa = _ensure_field(out_points_fc, "FA", "DOUBLE", fallbacks=("FLOWACC","FLOW_ACC"))

        with arcpy.da.SearchCursor(tmp_sorted, [id_on_pts, val_field, "SHAPE@"]) as cur, \
             arcpy.da.InsertCursor(out_points_fc, [out_id, out_fa, "SHAPE@"]) as ic:
            counts = {}
            for cw, fa, geom in cur:
                if cw is None:
                    continue
                cw = int(cw)
                counts.setdefault(cw, 0)
                if counts[cw] < int(top_n):
                    ic.insertRow((cw, float(fa) if fa is not None else None, geom))
                    counts[cw] += 1

    for p in [tmp_line, tmp_vtx, tmp_join, tmp_vals, tmp_keep, tmp_sorted]:
        _safe_delete(p)

    return out_points_fc

# ============================================================
# 0) Build projected masks ONCE
# ============================================================
inStreamsWS_tgt = os.path.join(ws_gdb, "inStreamsWS_tgt")
fix_define_and_project_to_gdb(inStreamsWatershed, inStreamsWS_tgt, D8_SR, name="inStreamsWS")
gp("RepairGeometry stream mask", arcpy.management.RepairGeometry, inStreamsWS_tgt)

inStreams_single = os.path.join(ws_gdb, "inStreamsWS_single")
_safe_delete(inStreams_single)
gp("MultipartToSinglepart stream mask", arcpy.management.MultipartToSinglepart, inStreamsWS_tgt, inStreams_single)

inStreamsWS_tgt_diss = os.path.join(ws_gdb, "inStreamsWS_tgt_diss")
_safe_delete(inStreamsWS_tgt_diss)
gp("Dissolve stream mask", arcpy.management.Dissolve, inStreams_single, inStreamsWS_tgt_diss)

inStreamsWS_buf = os.path.join(ws_gdb, f"inStreamsWS_buf{stream_buf_m}m")
_safe_delete(inStreamsWS_buf)
gp("Buffer stream mask", arcpy.analysis.Buffer, inStreamsWS_tgt_diss, inStreamsWS_buf,
   f"{stream_buf_m} Meters", "FULL", "ROUND", "ALL")

Lake_tgt = os.path.join(ws_gdb, "LakeHuron_tgt")
fix_define_and_project_to_gdb(Lake_Huron, Lake_tgt, D8_SR, assumed_src_if_mislabeled=arcpy.SpatialReference(4326), name="LakeHuron")
gp("RepairGeometry lake", arcpy.management.RepairGeometry, Lake_tgt)

lake_mask_ras = os.path.join(ws_gdb, "LakeHuron_mask_ras")
polygon_to_mask_raster(Lake_tgt, lake_mask_ras, value=1)

flowacc_land = os.path.join(ws_gdb, "flowacc_land")
_safe_delete(flowacc_land)
_log("▶ Build flowacc_land = flowacc where NOT lake")
sa.SetNull(sa.Raster(lake_mask_ras), sa.Raster(flowacc)).save(flowacc_land)
_log(f"✅ flowacc_land: {flowacc_land}")

# ============================================================
# MAIN LOOP
# ============================================================
for cat, (wet_fc, out_ws_drain, out_ws_lake) in cats.items():
    _log(f"\n==================== {cat.upper()} ====================")

    # 1) Project wetlands to D8 CRS
    wet_tgt = os.path.join(ws_gdb, f"{cat}_wet_tgt")
    _safe_delete(wet_tgt)
    gp(f"[{cat}] Project wetlands", arcpy.management.Project, wet_fc, wet_tgt, D8_SR)

    wet_id_f = _ensure_field(wet_tgt, CW_ID_FIELD, "LONG", fallbacks=("CWID","CW_ID","CW_Id"))
    gp(f"[{cat}] RepairGeometry wetlands", arcpy.management.RepairGeometry, wet_tgt)

    orig_ids = _idset(wet_tgt, wet_id_f)
    _log(f"[{cat}] unique CW_Ids in ORIGINAL wetlands: {len(orig_ids)}")

    # 2) Wetlands: remove ONLY overlapped part with stream mask (partial erase)
    wet_no_stream = os.path.join(ws_gdb, f"{cat}_wet_no_stream")
    _safe_delete(wet_no_stream)
    gp(f"[{cat}] Erase stream-mask from wetlands (PARTIAL)", arcpy.analysis.Erase, wet_tgt, inStreamsWS_buf, wet_no_stream)

    # 3) Land-only wetlands for outlets (also remove lake)
    wet_land = os.path.join(ws_gdb, f"{cat}_wet_land")
    _safe_delete(wet_land)
    gp(f"[{cat}] Erase lake from wetlands (land-only)", arcpy.analysis.Erase, wet_no_stream, Lake_tgt, wet_land)

    # 4) Primary pourpoints from wet_land boundary (Option A)
    pp_inside = os.path.join(pp_gdb, f"{cat}_pp_boundary_top{TOP_N_PER_CWID}")
    _safe_delete(pp_inside)
    gp(f"[{cat}] Build pourpoints (boundary from wet_land)", pourpoints_boundary_topN_flowacc_or_boundary,
       wet_land, pp_inside, wet_id_f, flowacc_land, ws_gdb, TOP_N_PER_CWID)

    # 4b) Fallback boundary pourpoints for CW_Ids that disappeared after wet_land erases
    pp_ids = _idset(pp_inside, wet_id_f)
    missing_for_pp = sorted(list(orig_ids - pp_ids))
    _log(f"[{cat}] CW_Ids missing pourpoints from wet_land: {len(missing_for_pp)}")

    if missing_for_pp:
        wet_orig_diss = os.path.join(ws_gdb, f"{cat}_wet_orig_diss")
        _safe_delete(wet_orig_diss)
        gp(f"[{cat}] Dissolve ORIGINAL wetlands by CW_Id (fallback pourpoints)", arcpy.management.Dissolve, wet_tgt, wet_orig_diss, wet_id_f)

        wet_missing_poly = os.path.join(ws_gdb, f"{cat}_wet_missing_poly")
        _safe_delete(wet_missing_poly)

        lyr = f"lyr_{cat}_miss_poly"
        arcpy.management.MakeFeatureLayer(wet_orig_diss, lyr)
        arcpy.management.SelectLayerByAttribute(lyr, "CLEAR_SELECTION")
        chunk = 900
        for i in range(0, len(missing_for_pp), chunk):
            sub = missing_for_pp[i:i+chunk]
            where = f"{arcpy.AddFieldDelimiters(lyr, wet_id_f)} IN ({','.join(map(str, sub))})"
            arcpy.management.SelectLayerByAttribute(lyr, "ADD_TO_SELECTION", where)
        gp(f"[{cat}] Copy missing CW_Id polygons (fallback)", arcpy.management.CopyFeatures, lyr, wet_missing_poly)
        arcpy.management.Delete(lyr)

        pp_fallback = os.path.join(pp_gdb, f"{cat}_pp_fallback_boundary")
        _safe_delete(pp_fallback)
        gp(f"[{cat}] Build fallback boundary pourpoints (missing IDs)", pourpoints_boundary_topN_flowacc_or_boundary,
           wet_missing_poly, pp_fallback, wet_id_f, flowacc_land, ws_gdb, TOP_N_PER_CWID)

        gp(f"[{cat}] Append fallback -> main pourpoints", arcpy.management.Append, pp_fallback, pp_inside, "NO_TEST")

        for p in [wet_orig_diss, wet_missing_poly, pp_fallback]:
            _safe_delete(p)

    pp_id_f = _ensure_field(pp_inside, wet_id_f, "LONG", fallbacks=("CWID","CW_ID","CW_Id"))
    _log(f"[{cat}] pourpoints CW_Ids present (after fallback): {count_unique(pp_inside, pp_id_f)}")

    # 5) Pass1 SnapPourPoint
    pp_ras = os.path.join(pp_gdb, f"{cat}_pp_ras")
    _safe_delete(pp_ras)
    gp(f"[{cat}] PointToRaster pourpoints (CW_Id)", arcpy.conversion.PointToRaster,
       pp_inside, pp_id_f, pp_ras, "MAXIMUM", "", cellsize)

    snapped_pp_ras1 = os.path.join(pp_gdb, f"{cat}_pp_snapped_ras1")
    _safe_delete(snapped_pp_ras1)
    _log(f"[{cat}] SnapPourPoint pass1 = {snap_dist_1} m")
    sa.SnapPourPoint(sa.Raster(pp_ras), sa.Raster(flowacc_land), snap_dist_1).save(snapped_pp_ras1)

    snapped_pts1 = os.path.join(pp_gdb, f"{cat}_pp_snapped_pts1")
    _safe_delete(snapped_pts1)
    gp(f"[{cat}] RasterToPoint snapped pourpoints (pass1)", arcpy.conversion.RasterToPoint, snapped_pp_ras1, snapped_pts1, "VALUE")

    val_field1 = _find_field(snapped_pts1, ["GRID_CODE", "GRIDCODE", "VALUE"])
    if not val_field1:
        raise RuntimeError(f"[{cat}] Could not find VALUE/GRIDCODE in snapped points pass1.")

    pp_final_id1 = _ensure_field(snapped_pts1, wet_id_f, "LONG", fallbacks=("CWID","CW_ID","CW_Id"))
    gp(f"[{cat}] Calculate CW_Id on snapped points (pass1)", arcpy.management.CalculateField,
       snapped_pts1, pp_final_id1, f"!{val_field1}!", "PYTHON3")

    snapped_ids1 = _idset(snapped_pts1, pp_final_id1)
    missing_after_snap1 = sorted(list(orig_ids - snapped_ids1))
    _log(f"[{cat}] snapped CW_Ids pass1: {len(snapped_ids1)} | missing after pass1: {len(missing_after_snap1)}")

    # 6) Pass2 SnapPourPoint ONLY for missing CW_Ids (bigger distance), append
    snapped_pts = os.path.join(pp_gdb, f"{cat}_pp_snapped_pts")
    _safe_delete(snapped_pts)
    gp(f"[{cat}] Copy snapped pass1 -> combined", arcpy.management.CopyFeatures, snapped_pts1, snapped_pts)

    if missing_after_snap1:
        miss_pp = os.path.join(pp_gdb, f"{cat}_pp_missing_for_snap2")
        _safe_delete(miss_pp)

        lyr = f"lyr_{cat}_ppinside"
        arcpy.management.MakeFeatureLayer(pp_inside, lyr)
        arcpy.management.SelectLayerByAttribute(lyr, "CLEAR_SELECTION")
        chunk = 900
        for i in range(0, len(missing_after_snap1), chunk):
            sub = missing_after_snap1[i:i+chunk]
            where = f"{arcpy.AddFieldDelimiters(lyr, pp_id_f)} IN ({','.join(map(str, sub))})"
            arcpy.management.SelectLayerByAttribute(lyr, "ADD_TO_SELECTION", where)
        gp(f"[{cat}] Copy missing pourpoints for snap2", arcpy.management.CopyFeatures, lyr, miss_pp)
        arcpy.management.Delete(lyr)

        miss_ras = os.path.join(pp_gdb, f"{cat}_pp_missing_ras")
        _safe_delete(miss_ras)
        gp(f"[{cat}] PointToRaster missing pourpoints", arcpy.conversion.PointToRaster,
           miss_pp, pp_id_f, miss_ras, "MAXIMUM", "", cellsize)

        snapped_pp_ras2 = os.path.join(pp_gdb, f"{cat}_pp_snapped_ras2")
        _safe_delete(snapped_pp_ras2)
        _log(f"[{cat}] SnapPourPoint pass2 = {snap_dist_2} m (missing CW_Ids only)")
        sa.SnapPourPoint(sa.Raster(miss_ras), sa.Raster(flowacc_land), snap_dist_2).save(snapped_pp_ras2)

        snapped_pts2 = os.path.join(pp_gdb, f"{cat}_pp_snapped_pts2")
        _safe_delete(snapped_pts2)
        gp(f"[{cat}] RasterToPoint snapped pourpoints (pass2)", arcpy.conversion.RasterToPoint, snapped_pp_ras2, snapped_pts2, "VALUE")

        val_field2 = _find_field(snapped_pts2, ["GRID_CODE", "GRIDCODE", "VALUE"])
        if not val_field2:
            raise RuntimeError(f"[{cat}] Could not find VALUE/GRIDCODE in snapped points pass2.")

        pp_final_id2 = _ensure_field(snapped_pts2, wet_id_f, "LONG", fallbacks=("CWID","CW_ID","CW_Id"))
        gp(f"[{cat}] Calculate CW_Id on snapped points (pass2)", arcpy.management.CalculateField,
           snapped_pts2, pp_final_id2, f"!{val_field2}!", "PYTHON3")

        gp(f"[{cat}] Append snapped pass2 -> combined", arcpy.management.Append, snapped_pts2, snapped_pts, "NO_TEST")

        for p in [miss_pp, miss_ras, snapped_pp_ras2, snapped_pts2]:
            _safe_delete(p)

    snapped_ids = _idset(snapped_pts, wet_id_f)
    _log(f"[{cat}] combined snapped CW_Ids: {len(snapped_ids)} (target {len(orig_ids)})")

    # ------------------------------------------------------------
    # COLLISION REPAIR (critical):
    # If multiple CW_Ids snapped to the SAME raster cell, PointToRaster will drop IDs.
    # Fix by re-snapping duplicates (only duplicates) and replacing them.
    # ------------------------------------------------------------
    if COLLISION_REPAIR:
        xy = ["SHAPE@X", "SHAPE@Y"]
        idfld = wet_id_f

        cell_to_ids = {}
        with arcpy.da.SearchCursor(snapped_pts, [idfld] + xy) as cur:
            for cw, x, y in cur:
                if cw is None:
                    continue
                key = (round(x, 3), round(y, 3))
                cell_to_ids.setdefault(key, []).append(int(cw))

        dupe_ids = []
        for key, ids in cell_to_ids.items():
            if len(ids) > 1:
                dupe_ids.extend(ids[1:])  # keep first, fix rest

        _log(f"[{cat}] duplicate snapped-cell CW_Ids: {len(dupe_ids)}")

        if dupe_ids:
            dup_src = os.path.join(pp_gdb, f"{cat}_pp_dupe_src")
            _safe_delete(dup_src)

            lyr = f"lyr_{cat}_pp_dupes"
            arcpy.management.MakeFeatureLayer(pp_inside, lyr)
            arcpy.management.SelectLayerByAttribute(lyr, "CLEAR_SELECTION")
            chunk = 900
            for i in range(0, len(dupe_ids), chunk):
                sub = dupe_ids[i:i+chunk]
                where = f"{arcpy.AddFieldDelimiters(lyr, pp_id_f)} IN ({','.join(map(str, sub))})"
                arcpy.management.SelectLayerByAttribute(lyr, "ADD_TO_SELECTION", where)
            gp(f"[{cat}] Copy duplicate CW_Id pourpoints", arcpy.management.CopyFeatures, lyr, dup_src)
            arcpy.management.Delete(lyr)

            dup_ras = os.path.join(pp_gdb, f"{cat}_pp_dupe_ras")
            _safe_delete(dup_ras)
            gp(f"[{cat}] PointToRaster dupe pourpoints", arcpy.conversion.PointToRaster,
               dup_src, pp_id_f, dup_ras, "MAXIMUM", "", cellsize)

            dup_snap_ras = os.path.join(pp_gdb, f"{cat}_pp_dupe_snapped_ras")
            _safe_delete(dup_snap_ras)
            dup_dist = max(COLLISION_RESNAP_DIST, snap_dist_1)
            _log(f"[{cat}] Re-snap duplicates with distance = {dup_dist} m")
            sa.SnapPourPoint(sa.Raster(dup_ras), sa.Raster(flowacc_land), dup_dist).save(dup_snap_ras)

            dup_snap_pts = os.path.join(pp_gdb, f"{cat}_pp_dupe_snapped_pts")
            _safe_delete(dup_snap_pts)
            gp(f"[{cat}] RasterToPoint dupe snapped", arcpy.conversion.RasterToPoint, dup_snap_ras, dup_snap_pts, "VALUE")

            vfld = _find_field(dup_snap_pts, ["GRID_CODE","GRIDCODE","VALUE"])
            dup_id = _ensure_field(dup_snap_pts, wet_id_f, "LONG")
            gp(f"[{cat}] Calc CW_Id dupe snapped", arcpy.management.CalculateField, dup_snap_pts, dup_id, f"!{vfld}!", "PYTHON3")

            snapped_clean = os.path.join(pp_gdb, f"{cat}_snapped_clean")
            _safe_delete(snapped_clean)

            lyr2 = f"lyr_{cat}_snapped"
            arcpy.management.MakeFeatureLayer(snapped_pts, lyr2)
            arcpy.management.SelectLayerByAttribute(lyr2, "CLEAR_SELECTION")  # start with all
            # remove duplicates from selection => select all EXCEPT dupes
            # easiest: select dupes then switch selection
            chunk = 900
            for i in range(0, len(dupe_ids), chunk):
                sub = dupe_ids[i:i+chunk]
                where = f"{arcpy.AddFieldDelimiters(lyr2, wet_id_f)} IN ({','.join(map(str, sub))})"
                arcpy.management.SelectLayerByAttribute(lyr2, "ADD_TO_SELECTION", where)
            arcpy.management.SelectLayerByAttribute(lyr2, "SWITCH_SELECTION")
            gp(f"[{cat}] Copy snapped (without dupes)", arcpy.management.CopyFeatures, lyr2, snapped_clean)
            arcpy.management.Delete(lyr2)

            gp(f"[{cat}] Append re-snapped dupes", arcpy.management.Append, dup_snap_pts, snapped_clean, "NO_TEST")

            _safe_delete(snapped_pts)
            gp(f"[{cat}] Replace snapped_pts with de-collided version", arcpy.management.CopyFeatures, snapped_clean, snapped_pts)

            for p in [dup_src, dup_ras, dup_snap_ras, dup_snap_pts, snapped_clean]:
                _safe_delete(p)

        snapped_ids = _idset(snapped_pts, wet_id_f)
        _log(f"[{cat}] snapped CW_Ids AFTER collision repair: {len(snapped_ids)} (target {len(orig_ids)})")

    # 7) Snapped points -> raster for Watershed (after collision repair)
    pp_snap_ras = os.path.join(pp_gdb, f"{cat}_pp_snap_ras")
    _safe_delete(pp_snap_ras)
    gp(f"[{cat}] PointToRaster snapped points (CW_Id)", arcpy.conversion.PointToRaster,
       snapped_pts, wet_id_f, pp_snap_ras, "MAXIMUM", "", cellsize)

    # 8) Watershed raster
    ws_ras = os.path.join(ws_gdb, f"{cat}_ws_ras")
    _safe_delete(ws_ras)
    _log(f"▶ [{cat}] Watershed")
    t0 = time.time()
    sa.Watershed(D8_flow, pp_snap_ras).save(ws_ras)
    _log(f"✅ DONE [{cat}] Watershed ({(time.time()-t0)/60:.2f} min)")

    # 9) RasterToPolygon + dissolve by CW_Id
    ws_poly_raw = os.path.join(ws_gdb, f"{cat}_ws_poly_raw")
    _safe_delete(ws_poly_raw)
    gp(f"[{cat}] RasterToPolygon", arcpy.conversion.RasterToPolygon, ws_ras, ws_poly_raw, "NO_SIMPLIFY", "VALUE")

    grid_field = _find_field(ws_poly_raw, ["GRIDCODE", "GRID_CODE"])
    if not grid_field:
        raise RuntimeError(f"[{cat}] GRIDCODE missing in watershed polygons.")

    ws_id_f = _ensure_field(ws_poly_raw, wet_id_f, "LONG", fallbacks=("CWID","CW_ID","CW_Id"))
    gp(f"[{cat}] Calculate CW_Id on watershed polygons", arcpy.management.CalculateField,
       ws_poly_raw, ws_id_f, f"!{grid_field}!", "PYTHON3")

    ws_poly = os.path.join(ws_gdb, f"{cat}_ws_poly")
    _safe_delete(ws_poly)
    gp(f"[{cat}] Dissolve watersheds by CW_Id", arcpy.management.Dissolve, ws_poly_raw, ws_poly, ws_id_f)

    ws_before_ids = _idset(ws_poly, ws_id_f)
    _log(f"[{cat}] watersheds BEFORE clip: {len(ws_before_ids)} CW_Ids (target {len(orig_ids)})")

    # 10) Clip out ONLY overlap parts (lake + stream mask)
    tmp_no_lake = os.path.join(ws_gdb, f"{cat}_ws_no_lake")
    tmp_no_stream = os.path.join(ws_gdb, f"{cat}_ws_no_stream")
    _safe_delete(tmp_no_lake); _safe_delete(tmp_no_stream)

    gp(f"[{cat}] Erase lake from watersheds (partial)", arcpy.analysis.Erase, ws_poly, Lake_tgt, tmp_no_lake)
    gp(f"[{cat}] Erase stream mask from watersheds (partial)", arcpy.analysis.Erase, tmp_no_lake, inStreamsWS_buf, tmp_no_stream)

    _safe_delete(out_ws_lake)
    gp(f"[{cat}] Dissolve final output (after clips)", arcpy.management.Dissolve, tmp_no_stream, out_ws_lake, ws_id_f)

    final_ids = _idset(out_ws_lake, ws_id_f)
    missing_after_clip = sorted(list(orig_ids - final_ids))
    _log(f"[{cat}] final watersheds AFTER clip: {len(final_ids)} CW_Ids (target {len(orig_ids)})")
    _log(f"[{cat}] missing CW_Ids after clip: {len(missing_after_clip)}")

    # 10b) OPTIONAL: fallback if clip erased entire polygon (keeps CW_Id coverage)
    if CLIP_FALLBACK and missing_after_clip:
        ws_missing = os.path.join(ws_gdb, f"{cat}_ws_missing_unclipped")
        _safe_delete(ws_missing)

        lyr = f"lyr_{cat}_ws_poly"
        arcpy.management.MakeFeatureLayer(ws_poly, lyr)
        arcpy.management.SelectLayerByAttribute(lyr, "CLEAR_SELECTION")
        chunk = 900
        for i in range(0, len(missing_after_clip), chunk):
            sub = missing_after_clip[i:i+chunk]
            where = f"{arcpy.AddFieldDelimiters(lyr, ws_id_f)} IN ({','.join(map(str, sub))})"
            arcpy.management.SelectLayerByAttribute(lyr, "ADD_TO_SELECTION", where)
        gp(f"[{cat}] Copy missing CW_Ids from UNCLIPPED watersheds (fallback)", arcpy.management.CopyFeatures, lyr, ws_missing)
        arcpy.management.Delete(lyr)

        gp(f"[{cat}] Append unclipped fallback -> final", arcpy.management.Append, ws_missing, out_ws_lake, "NO_TEST")

        out_ws_lake_diss = os.path.join(ws_gdb, f"{cat}_final_diss_after_fallback")
        _safe_delete(out_ws_lake_diss)
        if hasattr(arcpy.analysis, "PairwiseDissolve"):
            gp(f"[{cat}] PairwiseDissolve (enforce 1 per CW_Id)", arcpy.analysis.PairwiseDissolve, out_ws_lake, out_ws_lake_diss, ws_id_f)
        else:
            gp(f"[{cat}] Dissolve (enforce 1 per CW_Id)", arcpy.management.Dissolve, out_ws_lake, out_ws_lake_diss, ws_id_f)

        _safe_delete(out_ws_lake)
        gp(f"[{cat}] Copy final (with fallback)", arcpy.management.CopyFeatures, out_ws_lake_diss, out_ws_lake)

        final_ids = _idset(out_ws_lake, ws_id_f)
        _log(f"[{cat}] final CW_Ids AFTER clip-fallback: {len(final_ids)} (target {len(orig_ids)})")

        _safe_delete(ws_missing)
        _safe_delete(out_ws_lake_diss)

    # 11) Add attributes
    calculate_area_m2(out_ws_lake, "WS_AREAM2")
    add_xy_ll(out_ws_lake, prefix="WS")

    _log(f"✅ final watershed: {cat} -> {os.path.basename(out_ws_lake)}")
    _clear_locks()

_log("\n🎉 DONE")


✅ flowacc: S:\Projects\Active\GLB_Nutrient_Transport\DEM_rasters\GLB_Bdry_buff10km_dem_fill_flowaccu.tif
✅ workspace: D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\CoastalWatersheds\Watershed_rasters\watersheds.gdb
✅ scratch:   D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\CoastalWatersheds\Watershed_rasters\watersheds.gdb
✅ D8_flow CRS: NAD_1983_Great_Lakes_Basin_Albers (factoryCode=3174)

[inStreamsWS] input: D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Streamwatershed\PointWaterdhed_LH.shp
[inStreamsWS] sr: GCS_WGS_1984 | factoryCode=4326 | type=Geographic
[inStreamsWS] extent: XMin=929846.850 XMax=1166434.867 YMin=651452.122 YMax=1020243.911
▶ CopyFeatures inStreamsWS
✅ DONE CopyFeatures inStreamsWS (0.01 min)
[inStreamsWS] ⚠️ MISLABELED Geographic but coords are projected. DefineProjection -> NAD_1983_Great_Lakes_Basin_Albers
▶ DefineProjection inStreamsWS
✅ DONE DefineProjection inStreamsWS (0.00 min)
▶ Copy to output inStreamsWS
✅ DONE Copy to output inStreams

ExecuteError: ERROR 000464: Cannot get exclusive schema lock.  Either being edited or in use by another application or service.
Failed to execute (Append).
